In [27]:
# Locate and load a file named like 'combined_players_stats' into a DataFrame
import os
import glob
import pandas as pd

# Search common extensions in current working directory (recursively)
patterns = [
    "**/combined_players_stats.*",
    "**/combined_player_stats.*",
    "**/combined_players*stats.*",
]
files = []
for pat in patterns:
    files.extend(glob.glob(pat, recursive=True))

# Remove directories, keep unique files
files = sorted({f for f in files if os.path.isfile(f)})
print("Candidate files:\n", "\n ".join(files) if files else "<none found>")

# Try reading the first candidate if available
combined_players_stats = None
if files:
    fp = files[0]
    ext = os.path.splitext(fp)[1].lower()
    print(f"\nReading: {fp} (ext={ext})")
    if ext in [".csv"]:
        combined_players_stats = pd.read_csv(fp)
    elif ext in [".parquet"]:
        combined_players_stats = pd.read_parquet(fp)
    elif ext in [".xlsx", ".xls"]:
        combined_players_stats = pd.read_excel(fp)
    elif ext in [".feather"]:
        combined_players_stats = pd.read_feather(fp)
    else:
        raise ValueError(f"Unsupported extension for auto-load: {ext}. Found file: {fp}")

    print("\nLoaded DataFrame: combined_players_stats")
    print("Shape:", combined_players_stats.shape)
    display(combined_players_stats.head())
else:
    print("\nNo matching file found. If this is a variable in a previous notebook cell, please share how it was created, or provide the filename/path.")



    


Candidate files:
 combined_players_stats.csv

Reading: combined_players_stats.csv (ext=.csv)

Loaded DataFrame: combined_players_stats
Shape: (3552, 37)


,Player,Nation,Pos,Age,Playing Time MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG,Matches,League,Team,MP
0,William Saliba,fr FRA,DF,22.0,38.0,38,3420.0,38.0,2.0,1.0,...,0.08,0.04,0.01,0.05,0.04,0.05,Matches,EPL,Arsenal,NaN
1,Declan Rice,eng ENG,MF,24.0,38.0,37,3225.0,35.8,7.0,8.0,...,0.42,0.09,0.15,0.24,0.09,0.24,Matches,EPL,Arsenal,NaN
2,Martin Ødegaard,no NOR,MF,24.0,35.0,35,3091.0,34.3,8.0,10.0,...,0.47,0.22,0.28,0.50,0.17,0.45,Matches,EPL,Arsenal,NaN
3,Ben White,eng ENG,DF,25.0,37.0,35,2988.0,33.2,4.0,4.0,...,0.24,0.03,0.11,0.15,0.03,0.15,Matches,EPL,Arsenal,NaN
4,Bukayo Saka,eng ENG,FW,21.0,35.0,35,2919.0,32.4,16.0,9.0,...,0.59,0.48,0.32,0.80,0.33,0.65,Matches,EPL,Arsenal,NaN


In [28]:
# Drop the 'Team' and 'League' columns from combined_players_stats
cols_to_drop = [c for c in ["Team", "League"] if c in combined_players_stats.columns]
combined_players_stats = combined_players_stats.drop(columns=cols_to_drop)

print("Dropped columns:", cols_to_drop)
print("New shape:", combined_players_stats.shape)
print("Remaining columns (first 15):", list(combined_players_stats.columns[:15]))
display(combined_players_stats.head())

Dropped columns: ['Team', 'League']
New shape: (3552, 35)
Remaining columns (first 15): ['Player', 'Nation', 'Pos', 'Age', 'Playing Time MP', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s', 'Performance Gls', 'Performance Ast', 'Performance G+A', 'Performance G-PK', 'Performance PK', 'Performance PKatt', 'Performance CrdY']


,Player,Nation,Pos,Age,Playing Time MP,Playing Time Starts,Playing Time Min,Playing Time 90s,Performance Gls,Performance Ast,...,Per 90 Minutes G+A,Per 90 Minutes G-PK,Per 90 Minutes G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,Per 90 Minutes xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG,Matches,MP
0,William Saliba,fr FRA,DF,22.0,38.0,38,3420.0,38.0,2.0,1.0,...,0.08,0.05,0.08,0.04,0.01,0.05,0.04,0.05,Matches,NaN
1,Declan Rice,eng ENG,MF,24.0,38.0,37,3225.0,35.8,7.0,8.0,...,0.42,0.20,0.42,0.09,0.15,0.24,0.09,0.24,Matches,NaN
2,Martin Ødegaard,no NOR,MF,24.0,35.0,35,3091.0,34.3,8.0,10.0,...,0.52,0.17,0.47,0.22,0.28,0.50,0.17,0.45,Matches,NaN
3,Ben White,eng ENG,DF,25.0,37.0,35,2988.0,33.2,4.0,4.0,...,0.24,0.12,0.24,0.03,0.11,0.15,0.03,0.15,Matches,NaN
4,Bukayo Saka,eng ENG,FW,21.0,35.0,35,2919.0,32.4,16.0,9.0,...,0.77,0.31,0.59,0.48,0.32,0.80,0.33,0.65,Matches,NaN


In [29]:
# Inspect and normalize the Nation field so we can filter by allowed countries
# FBref-style Nation often looks like 'eng ENG' (lowercase tag + uppercase FIFA trigram)

nation_raw = combined_players_stats['Nation'].astype(str)
combined_players_stats['Nation_code'] = nation_raw.str.extract(r'([A-Z]{3})', expand=False)
combined_players_stats['Nation_tag'] = nation_raw.str.extract(r'^([a-z]{2,3})', expand=False)

print('Nation raw sample:')
display(combined_players_stats[['Nation','Nation_tag','Nation_code']].head(10))

print('Unique Nation_code count:', combined_players_stats['Nation_code'].nunique(dropna=True))
print('Top 20 Nation_code values:')
print(combined_players_stats['Nation_code'].value_counts(dropna=False).head(20))


Nation raw sample:


,Nation,Nation_tag,Nation_code
0,fr FRA,fr,FRA
1,eng ENG,eng,ENG
2,no NOR,no,NOR
3,eng ENG,eng,ENG
4,eng ENG,eng,ENG
5,br BRA,br,BRA
6,es ESP,es,ESP
7,de GER,de,GER
8,br BRA,br,BRA
9,ua UKR,ua,UKR


Unique Nation_code count: 112
Top 20 Nation_code values:
Nation_code
ESP    518
FRA    421
ITA    341
ENG    320
GER    314
BRA    112
ARG     84
POR     82
NED     67
BEL     62
CIV     52
SEN     50
MAR     49
SRB     49
DEN     45
SUI     39
POL     38
CRO     37
AUT     36
IRL     36
Name: count, dtype: int64


In [30]:
# Filter players to an allowed list of countries (by Nation_code), and report counts

allowed_country_names = [
    "Algeria","Argentina","Australia","Austria","Belgium","Brazil","Canada","Cape Verde","Colombia",
    "Croatia","Curacao","Ecuador","Egypt","England","France","Germany","Ghana","Haiti","IR Iran",
    "Ivory Coast","Japan","Jordan","Mexico","Morocco","Netherlands","New Zealand","Norway","Panama",
    "Paraguay","Portugal","Qatar","Saudi Arabia","Scotland","Senegal","South Africa","South Korea",
    "Spain","Switzerland","Tunisia","United States","Uruguay","Uzbekistan"
]

# Common FIFA/FBref 3-letter codes for the listed countries
name_to_code = {
    "Algeria": "ALG",
    "Argentina": "ARG",
    "Australia": "AUS",
    "Austria": "AUT",
    "Belgium": "BEL",
    "Brazil": "BRA",
    "Canada": "CAN",
    "Cape Verde": "CPV",
    "Colombia": "COL",
    "Croatia": "CRO",
    "Curacao": "CUW",
    "Ecuador": "ECU",
    "Egypt": "EGY",
    "England": "ENG",
    "France": "FRA",
    "Germany": "GER",
    "Ghana": "GHA",
    "Haiti": "HAI",
    "IR Iran": "IRN",
    "Ivory Coast": "CIV",
    "Japan": "JPN",
    "Jordan": "JOR",
    "Mexico": "MEX",
    "Morocco": "MAR",
    "Netherlands": "NED",
    "New Zealand": "NZL",
    "Norway": "NOR",
    "Panama": "PAN",
    "Paraguay": "PAR",
    "Portugal": "POR",
    "Qatar": "QAT",
    "Saudi Arabia": "KSA",
    "Scotland": "SCO",
    "Senegal": "SEN",
    "South Africa": "RSA",
    "South Korea": "KOR",
    "Spain": "ESP",
    "Switzerland": "SUI",
    "Tunisia": "TUN",
    "United States": "USA",
    "Uruguay": "URU",
    "Uzbekistan": "UZB",
}

allowed_codes = sorted({name_to_code.get(n) for n in allowed_country_names if name_to_code.get(n) is not None})
missing = sorted([n for n in allowed_country_names if n not in name_to_code])

before_n = len(combined_players_stats)
filtered_players_stats = combined_players_stats[combined_players_stats["Nation_code"].isin(allowed_codes)].copy()
after_n = len(filtered_players_stats)

print(f"Allowed countries: {len(allowed_country_names)}")
print(f"Allowed codes mapped: {len(allowed_codes)}")
print("Missing name->code mappings:", missing if missing else "<none>")
print(f"Rows before: {before_n:,} | after: {after_n:,} | removed: {before_n - after_n:,}")

print("\nNation_code distribution (kept; top 20):")
print(filtered_players_stats["Nation_code"].value_counts().head(20))

display(filtered_players_stats[["Player","Nation","Nation_code","Pos","Age"]].head(10))


Allowed countries: 42
Allowed codes mapped: 42
Missing name->code mappings: <none>
Rows before: 3,552 | after: 2,541 | removed: 1,011

Nation_code distribution (kept; top 20):
Nation_code
ESP    518
FRA    421
ENG    320
GER    314
BRA    112
ARG     84
POR     82
NED     67
BEL     62
CIV     52
SEN     50
MAR     49
SUI     39
CRO     37
AUT     36
URU     33
COL     32
ALG     29
NOR     28
USA     28
Name: count, dtype: int64


,Player,Nation,Nation_code,Pos,Age
0,William Saliba,fr FRA,FRA,DF,22.0
1,Declan Rice,eng ENG,ENG,MF,24.0
2,Martin Ødegaard,no NOR,NOR,MF,24.0
3,Ben White,eng ENG,ENG,DF,25.0
4,Bukayo Saka,eng ENG,ENG,FW,21.0
5,Gabriel Magalhães,br BRA,BRA,DF,25.0
6,David Raya,es ESP,ESP,GK,27.0
7,Kai Havertz,de GER,GER,"MF,FW",24.0
8,Gabriel Martinelli,br BRA,BRA,FW,22.0
10,Leandro Trossard,be BEL,BEL,"FW,MF",28.0


In [31]:
filtered_players_stats.columns

Index(['Player', 'Nation', 'Pos', 'Age', 'Playing Time MP',
       'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s',
       'Performance Gls', 'Performance Ast', 'Performance G+A',
       'Performance G-PK', 'Performance PK', 'Performance PKatt',
       'Performance CrdY', 'Performance CrdR', 'Expected xG', 'Expected npxG',
       'Expected xAG', 'Expected npxG+xAG', 'Progression PrgC',
       'Progression PrgP', 'Progression PrgR', 'Per 90 Minutes Gls',
       'Per 90 Minutes Ast', 'Per 90 Minutes G+A', 'Per 90 Minutes G-PK',
       'Per 90 Minutes G+A-PK', 'Per 90 Minutes xG', 'Per 90 Minutes xAG',
       'Per 90 Minutes xG+xAG', 'Per 90 Minutes npxG',
       'Per 90 Minutes npxG+xAG', 'Matches', 'MP', 'Nation_code',
       'Nation_tag'],
      dtype='object')

In [32]:
# Create a coarse position group (GK/DEF/MID/FWD) from the FBref 'Pos' column

def pos_to_group(pos_val: str) -> str:
    if pd.isna(pos_val):
        return pd.NA
    s = str(pos_val).upper()
    # FBref often uses: GK, DF, MF, FW and combos like 'MF,FW'
    parts = [p.strip() for p in s.replace(';', ',').split(',') if p.strip()]

    if 'GK' in parts:
        return 'GK'
    # If multiple positions, prefer more attacking role for simplicity
    if 'FW' in parts:
        return 'FWD'
    if 'MF' in parts:
        return 'MID'
    if 'DF' in parts:
        return 'DEF'
    return pd.NA

filtered_players_stats['Pos_group'] = filtered_players_stats['Pos'].apply(pos_to_group)

print('Pos_group value counts:')
print(filtered_players_stats['Pos_group'].value_counts(dropna=False))

print('\nSample rows by Pos_group:')
display(
    filtered_players_stats[['Player','Pos','Pos_group','Nation_code','Age','Playing Time 90s']]
    .sample(12, random_state=7)
    .sort_values('Pos_group')
)


Pos_group value counts:
Pos_group
FWD    855
DEF    744
MID    704
GK     238
Name: count, dtype: int64

Sample rows by Pos_group:


,Player,Pos,Pos_group,Nation_code,Age,Playing Time 90s
31,Reuell Walters,DF,DEF,ENG,18.0,0.0
2807,Anthony Rouault,DF,DEF,FRA,22.0,0.0
1947,Maxim Leitsch,DF,DEF,GER,25.0,4.2
3070,Gonçalo Paciência,FW,FWD,POR,28.0,0.0
590,Ethan Horvath,GK,GK,USA,28.0,0.0
3029,Nando Almodóvar,GK,GK,ESP,19.0,0.0
523,Sean Longstaff,MF,MID,ENG,25.0,30.5
1951,Anton Stach,MF,MID,GER,24.0,1.9
1177,Luis Alberto,MF,MID,ESP,30.0,25.7
459,Kevin De Bruyne,MF,MID,BEL,32.0,13.6


In [33]:
# Define position-based weights for selected per-90 and progression features (no scoring yet)
import numpy as np

# Feature columns to be used (existing in filtered_players_stats)
feature_cols = {
    "gls_p90": "Per 90 Minutes Gls",
    "ast_p90": "Per 90 Minutes Ast",
    "xg_p90": "Per 90 Minutes xG",
    "xag_p90": "Per 90 Minutes xAG",
    "prgc": "Progression PrgC",
    "prgp": "Progression PrgP",
    "prgr": "Progression PrgR",
    "crdy": "Performance CrdY",
    "crdr": "Performance CrdR",
}

# Stars → numeric weights (simple linear: 1★=1, 2★=2, 3★=3)
# Cards are handled separately as penalties (subtracted), so weights here are positive magnitudes.
weights_by_pos = {
    "GK": {  # not specified by user; set to 0 for these attacking/progression features by default
        "gls_p90": 0, "ast_p90": 0, "xg_p90": 0, "xag_p90": 0,
        "prgc": 0, "prgp": 0, "prgr": 0,
        "crdy": 1, "crdr": 1,
    },
    "DEF": {
        "gls_p90": 1,
        "ast_p90": 2,
        "xg_p90": 1,
        "xag_p90": 2,
        "prgc": 3,
        "prgp": 3,
        "prgr": 3,
        "crdy": 1,
        "crdr": 1,
    },
    "MID": {
        "gls_p90": 2,
        "ast_p90": 3,
        "xg_p90": 2,
        "xag_p90": 3,
        "prgc": 3,
        "prgp": 3,
        "prgr": 3,
        "crdy": 1,
        "crdr": 1,
    },
    "FWD": {
        "gls_p90": 3,
        "ast_p90": 2,
        "xg_p90": 3,
        "xag_p90": 2,
        "prgc": 2,
        "prgp": 2,
        "prgr": 2,
        "crdy": 1,
        "crdr": 1,
    },
}

# Quick column existence check
missing_cols = [col for col in feature_cols.values() if col not in filtered_players_stats.columns]
print("Missing required columns:", missing_cols if missing_cols else "<none>")

print("Available Pos_group values:", sorted(filtered_players_stats["Pos_group"].dropna().unique().tolist()))
print("Weights keys:", list(weights_by_pos.keys()))


Missing required columns: <none>
Available Pos_group values: ['DEF', 'FWD', 'GK', 'MID']
Weights keys: ['GK', 'DEF', 'MID', 'FWD']


In [34]:
# Compute a position-specific weighted score using the defined weights
# - Uses per-90 attacking metrics + progression metrics
# - Subtracts card contributions (CrdY/CrdR)
# - Uses (xG_p90 + npxG_p90) / 2 to represent "xG / npxG" as a single blended signal

import numpy as np

scoring_df = filtered_players_stats.copy()

# Ensure numeric dtype for scoring columns
num_cols = [
    feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], feature_cols["xag_p90"],
    feature_cols["prgc"], feature_cols["prgp"], feature_cols["prgr"],
    feature_cols["crdy"], feature_cols["crdr"],
    "Per 90 Minutes npxG",
]
for c in num_cols:
    scoring_df[c] = pd.to_numeric(scoring_df[c], errors="coerce")

# Blended xG/npxG per 90 (user requested xG / npxG)
scoring_df["xg_npxg_p90_blend"] = scoring_df[[feature_cols["xg_p90"], "Per 90 Minutes npxG"]].mean(axis=1)

# Helper to compute row-wise weighted score based on Pos_group
keys_positive = ["gls_p90", "ast_p90", "xg_p90", "xag_p90", "prgc", "prgp", "prgr"]

def compute_weighted_score(row):
    pos = row.get("Pos_group")
    if pd.isna(pos) or pos not in weights_by_pos:
        return np.nan

    w = weights_by_pos[pos]

    # Positive contributions
    gls = row[feature_cols["gls_p90"]]
    ast = row[feature_cols["ast_p90"]]
    xg_blend = row["xg_npxg_p90_blend"]
    xag = row[feature_cols["xag_p90"]]
    prgc = row[feature_cols["prgc"]]
    prgp = row[feature_cols["prgp"]]
    prgr = row[feature_cols["prgr"]]

    # Use the xG weight for the blended xG/npxG signal
    pos_score = (
        w["gls_p90"] * gls +
        w["ast_p90"] * ast +
        w["xg_p90"] * xg_blend +
        w["xag_p90"] * xag +
        w["prgc"] * prgc +
        w["prgp"] * prgp +
        w["prgr"] * prgr
    )

    # Card penalties (subtract)
    crdy = row[feature_cols["crdy"]]
    crdr = row[feature_cols["crdr"]]
    pos_score = pos_score - (w["crdy"] * crdy + w["crdr"] * crdr)

    return pos_score

scoring_df["weighted_score"] = scoring_df.apply(compute_weighted_score, axis=1)

print("Weighted score computed.")
print("Missing weighted_score:", scoring_df["weighted_score"].isna().mean())

display(
    scoring_df[["Player","Pos","Pos_group","Nation_code","Age","Playing Time 90s","weighted_score"]]
    .sort_values("weighted_score", ascending=False)
    .head(15)
)

print("\nTop 10 by position group:")
for g in ["FWD","MID","DEF","GK"]:
    sub = scoring_df[scoring_df["Pos_group"]==g].sort_values("weighted_score", ascending=False)
    print(f"\n{g} (n={len(sub)}):")
    display(sub[["Player","Nation_code","Age","Playing Time 90s","weighted_score",
                 feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], "Per 90 Minutes npxG",
                 feature_cols["xag_p90"], feature_cols["prgc"], feature_cols["prgp"], feature_cols["prgr"],
                 feature_cols["crdy"], feature_cols["crdr"]]].head(10))


Weighted score computed.
Missing weighted_score: 0.0


,Player,Pos,Pos_group,Nation_code,Age,Playing Time 90s,weighted_score
1612,Florian Wirtz,MF,MID,GER,20.0,26.4,2004.620
2,Martin Ødegaard,MF,MID,NOR,24.0,34.3,1911.560
147,Pascal Groß,"MF,DF",MID,GER,32.0,34.6,1691.180
1992,Xavi Simons,MF,MID,NED,20.0,29.5,1653.150
1611,Jeremie Frimpong,DF,DEF,NED,22.0,25.0,1645.710
2640,Achraf Hakimi,DF,DEF,MAR,24.0,21.5,1594.330
4,Bukayo Saka,FW,FWD,ENG,21.0,32.4,1577.885
446,Rodri,MF,MID,ESP,27.0,32.6,1513.940
1608,Granit Xhaka,MF,MID,SUI,30.0,31.3,1510.590
1674,Kevin Stöger,MF,MID,AUT,29.0,29.6,1503.800



Top 10 by position group:

FWD (n=855):


,Player,Nation_code,Age,Playing Time 90s,weighted_score,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,Progression PrgC,Progression PrgP,Progression PrgR,Performance CrdY,Performance CrdR
4,Bukayo Saka,ENG,21.0,32.4,1577.885,0.49,0.28,0.48,0.33,0.32,155.0,126.0,508.0,4.0,0.0
3116,Sávio,BRA,19.0,33.1,1289.430,0.27,0.30,0.18,0.18,0.24,181.0,93.0,372.0,5.0,0.0
373,Mohamed Salah,EGY,31.0,28.2,1175.430,0.64,0.36,0.75,0.55,0.42,98.0,147.0,341.0,2.0,0.0
641,Son Heung-min,KOR,31.0,32.6,1168.935,0.52,0.31,0.37,0.32,0.36,119.0,139.0,325.0,1.0,0.0
451,Bernardo Silva,POR,28.0,28.6,1148.180,0.21,0.31,0.13,0.13,0.27,140.0,177.0,260.0,8.0,0.0
457,Jeremy Doku,BEL,21.0,17.7,1139.350,0.17,0.45,0.14,0.14,0.26,218.0,57.0,295.0,3.0,0.0
1250,Rafael Leão,POR,24.0,27.9,1138.320,0.32,0.32,0.36,0.36,0.32,160.0,117.0,293.0,5.0,0.0
478,Bruno Fernandes,POR,28.0,34.6,1123.715,0.29,0.23,0.29,0.18,0.34,86.0,297.0,182.0,9.0,0.0
2022,Chris Führich,GER,25.0,28.5,1114.545,0.28,0.25,0.23,0.20,0.28,153.0,145.0,259.0,2.0,0.0
2964,Lamine Yamal,ESP,16.0,24.5,1105.130,0.20,0.20,0.21,0.21,0.25,123.0,80.0,350.0,3.0,0.0



MID (n=704):


,Player,Nation_code,Age,Playing Time 90s,weighted_score,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,Progression PrgC,Progression PrgP,Progression PrgR,Performance CrdY,Performance CrdR
1612,Florian Wirtz,GER,20.0,26.4,2004.62,0.42,0.42,0.34,0.31,0.29,138.0,224.0,306.0,3.0,0.0
2,Martin Ødegaard,NOR,24.0,34.3,1911.56,0.23,0.29,0.22,0.17,0.28,91.0,344.0,202.0,2.0,0.0
147,Pascal Groß,GER,32.0,34.6,1691.18,0.12,0.29,0.19,0.16,0.24,83.0,302.0,180.0,6.0,0.0
1992,Xavi Simons,NED,20.0,29.5,1653.15,0.27,0.37,0.28,0.23,0.33,151.0,187.0,216.0,11.0,1.0
446,Rodri,ESP,27.0,32.6,1513.94,0.25,0.28,0.12,0.12,0.12,76.0,376.0,55.0,8.0,1.0
1608,Granit Xhaka,SUI,30.0,31.3,1510.59,0.10,0.00,0.06,0.06,0.09,60.0,392.0,53.0,5.0,0.0
1674,Kevin Stöger,AUT,29.0,29.6,1503.80,0.24,0.34,0.21,0.10,0.33,81.0,283.0,139.0,8.0,0.0
3324,Isco,ESP,31.0,25.8,1375.52,0.31,0.19,0.32,0.26,0.25,100.0,216.0,145.0,10.0,0.0
3364,Jude Bellingham,ENG,20.0,25.7,1371.63,0.74,0.23,0.43,0.40,0.21,85.0,196.0,177.0,5.0,1.0
2367,Maxence Caqueret,FRA,23.0,33.6,1355.99,0.03,0.09,0.06,0.06,0.18,70.0,286.0,96.0,1.0,0.0



DEF (n=744):


,Player,Nation_code,Age,Playing Time 90s,weighted_score,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,Progression PrgC,Progression PrgP,Progression PrgR,Performance CrdY,Performance CrdR
1611,Jeremie Frimpong,NED,22.0,25.0,1645.710,0.36,0.28,0.33,0.33,0.23,153.0,49.0,348.0,6.0,0.0
2640,Achraf Hakimi,MAR,24.0,21.5,1594.330,0.19,0.23,0.20,0.20,0.24,84.0,218.0,230.0,3.0,0.0
1609,Álex Grimaldo,ESP,27.0,30.9,1500.940,0.32,0.42,0.18,0.18,0.30,77.0,162.0,261.0,1.0,0.0
1993,David Raum,GER,25.0,30.4,1357.090,0.07,0.26,0.02,0.02,0.24,77.0,131.0,246.0,6.0,0.0
2961,João Cancelo,POR,29.0,27.8,1313.850,0.07,0.14,0.06,0.06,0.22,106.0,141.0,193.0,7.0,0.0
337,Antonee Robinson,USA,25.0,36.3,1260.580,0.00,0.17,0.02,0.02,0.11,99.0,129.0,194.0,6.0,0.0
640,Pedro Porro,ESP,23.0,34.3,1225.000,0.09,0.20,0.09,0.09,0.21,57.0,194.0,158.0,3.0,0.0
3115,Miguel Gutiérrez,ESP,22.0,33.7,1209.000,0.06,0.21,0.08,0.08,0.22,74.0,146.0,184.0,4.0,0.0
450,Kyle Walker,ENG,33.0,30.7,1207.450,0.00,0.13,0.01,0.01,0.09,74.0,157.0,172.0,2.0,0.0
1248,Theo Hernández,FRA,25.0,31.0,1203.835,0.16,0.13,0.16,0.11,0.14,115.0,166.0,124.0,11.0,1.0



GK (n=238):


,Player,Nation_code,Age,Playing Time 90s,weighted_score,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,Progression PrgC,Progression PrgP,Progression PrgR,Performance CrdY,Performance CrdR
17,Aaron Ramsdale,ENG,25.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
64,Joe Gauci,AUS,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,Sam Proctor,ENG,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65,Filip Marshall,ENG,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105,Callan Mckenna,SCO,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,James Wright,ENG,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157,Jason Steele,ENG,32.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137,Vincent Angelini,SCO,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,Ellery Balcombe,ENG,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
406,Marcelo Pitaluga,BRA,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Create a normalized, position-weighted score that avoids domination by raw progression totals
# - Convert PrgC/PrgP/PrgR totals to per-90 equivalents using Playing Time 90s
# - Recompute weighted_score_norm using the same star weights
# - Provide basic summaries and top players per position (with a minutes filter)

import numpy as np

scoring_norm_df = filtered_players_stats.copy()

# Ensure numeric columns
cols_needed = [
    feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], feature_cols["xag_p90"],
    feature_cols["prgc"], feature_cols["prgp"], feature_cols["prgr"],
    feature_cols["crdy"], feature_cols["crdr"],
    "Per 90 Minutes npxG",
    "Playing Time 90s",
]
for c in cols_needed:
    scoring_norm_df[c] = pd.to_numeric(scoring_norm_df[c], errors="coerce")

# Per-90 progression (handle divide-by-zero safely)
minutes_90s = scoring_norm_df["Playing Time 90s"].replace({0: np.nan})
scoring_norm_df["PrgC_p90"] = scoring_norm_df[feature_cols["prgc"]] / minutes_90s
scoring_norm_df["PrgP_p90"] = scoring_norm_df[feature_cols["prgp"]] / minutes_90s
scoring_norm_df["PrgR_p90"] = scoring_norm_df[feature_cols["prgr"]] / minutes_90s

# Blend xG/npxG per 90
scoring_norm_df["xg_npxg_p90_blend"] = scoring_norm_df[[feature_cols["xg_p90"], "Per 90 Minutes npxG"]].mean(axis=1)

def compute_weighted_score_norm(row):
    pos = row.get("Pos_group")
    if pd.isna(pos) or pos not in weights_by_pos:
        return np.nan
    w = weights_by_pos[pos]

    # positive
    pos_score = (
        w["gls_p90"] * row[feature_cols["gls_p90"]] +
        w["ast_p90"] * row[feature_cols["ast_p90"]] +
        w["xg_p90"] * row["xg_npxg_p90_blend"] +
        w["xag_p90"] * row[feature_cols["xag_p90"]] +
        w["prgc"] * row["PrgC_p90"] +
        w["prgp"] * row["PrgP_p90"] +
        w["prgr"] * row["PrgR_p90"]
    )

    # subtract cards
    pos_score = pos_score - (w["crdy"] * row[feature_cols["crdy"]] + w["crdr"] * row[feature_cols["crdr"]])
    return pos_score

scoring_norm_df["weighted_score_norm"] = scoring_norm_df.apply(compute_weighted_score_norm, axis=1)

# Quick summaries
print("weighted_score_norm missing rate:", scoring_norm_df["weighted_score_norm"].isna().mean())
print("weighted_score_norm summary (all players):")
display(scoring_norm_df["weighted_score_norm"].describe())

# Minutes filter for more stable rankings
min_90s = 10
eligible = scoring_norm_df[scoring_norm_df["Playing Time 90s"] >= min_90s].copy()
print(f"\nEligible players with Playing Time 90s >= {min_90s}: {len(eligible):,} / {len(scoring_norm_df):,}")

print("\nTop 10 overall (eligible):")
display(
    eligible[["Player","Pos","Pos_group","Nation_code","Age","Playing Time 90s","weighted_score_norm"]]
    .sort_values("weighted_score_norm", ascending=False)
    .head(10)
)

for g in ["FWD","MID","DEF","GK"]:
    subg = eligible[eligible["Pos_group"]==g].sort_values("weighted_score_norm", ascending=False)
    print(f"\nTop 10 {g} (eligible; n={len(subg)}):")
    display(subg[[
        "Player","Nation_code","Age","Playing Time 90s","weighted_score_norm",
        feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], "Per 90 Minutes npxG",
        feature_cols["xag_p90"], "PrgC_p90","PrgP_p90","PrgR_p90",
        feature_cols["crdy"], feature_cols["crdr"],
    ]].head(10))


weighted_score_norm missing rate: 0.1951987406532861
weighted_score_norm summary (all players):


count    2045.000000
mean       22.203573
std        17.319925
min        -5.000000
25%        11.724151
50%        20.453846
75%        30.000000
max       257.870000
Name: weighted_score_norm, dtype: float64


Eligible players with Playing Time 90s >= 10: 1,201 / 2,541

Top 10 overall (eligible):


,Player,Pos,Pos_group,Nation_code,Age,Playing Time 90s,weighted_score_norm
1612,Florian Wirtz,MF,MID,GER,20.0,26.4,76.529091
2640,Achraf Hakimi,DF,DEF,MAR,24.0,21.5,72.562558
459,Kevin De Bruyne,MF,MID,BEL,32.0,13.6,67.901176
2645,Ousmane Dembélé,"MF,FW",FWD,FRA,26.0,16.7,67.750659
457,Jeremy Doku,"FW,MF",FWD,BEL,21.0,17.7,63.756780
1611,Jeremie Frimpong,DF,DEF,NED,22.0,25.0,61.710000
1649,Kingsley Coman,FW,FWD,FRA,27.0,12.4,59.060645
2927,Samuel Lino,"DF,MF",MID,BRA,23.0,23.1,56.954156
2,Martin Ødegaard,MF,MID,NOR,24.0,34.3,56.274286
2933,Rodrigo Riquelme,"DF,MF",MID,ESP,23.0,17.0,55.090000



Top 10 FWD (eligible; n=375):


...

,Player,Nation_code,Age,Playing Time 90s,weighted_score_norm,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,Performance CrdY,Performance CrdR
1612,Florian Wirtz,GER,20.0,26.4,76.529091,0.42,0.42,0.34,0.31,0.29,5.227273,8.484848,11.590909,3.0,0.0
459,Kevin De Bruyne,BEL,32.0,13.6,67.901176,0.29,0.74,0.18,0.18,0.85,3.455882,8.161765,9.779412,2.0,0.0
2927,Samuel Lino,BRA,23.0,23.1,56.954156,0.17,0.22,0.24,0.24,0.21,5.670996,3.636364,9.307359,1.0,0.0
2,Martin Ødegaard,NOR,24.0,34.3,56.274286,0.23,0.29,0.22,0.17,0.28,2.653061,10.029155,5.889213,2.0,0.0
2933,Rodrigo Riquelme,ESP,23.0,17.0,55.090000,0.18,0.29,0.16,0.16,0.18,5.117647,3.117647,9.764706,1.0,0.0
3373,Luka Modrić,CRO,37.0,18.8,53.517872,0.11,0.32,0.07,0.07,0.30,2.819149,10.425532,4.521277,2.0,0.0
647,James Maddison,ENG,26.0,23.7,53.213038,0.17,0.38,0.27,0.27,0.25,2.447257,9.451477,6.582278,5.0,0.0
3364,Jude Bellingham,ENG,20.0,25.7,51.093035,0.74,0.23,0.43,0.40,0.21,3.307393,7.626459,6.887160,5.0,1.0
1637,Joshua Kimmich,GER,28.0,24.2,49.292810,0.04,0.25,0.05,0.05,0.30,2.314050,9.462810,4.710744,1.0,1.0
1613,Jonas Hofmann,GER,31.0,24.5,49.086939,0.20,0.29,0.24,0.24,0.33,2.571429,4.857143,9.020408,3.0,0.0



Top 10 DEF (eligible; n=413):


,Player,Nation_code,Age,Playing Time 90s,weighted_score_norm,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,Performance CrdY,Performance CrdR
2640,Achraf Hakimi,MAR,24.0,21.5,72.562558,0.19,0.23,0.20,0.20,0.24,3.906977,10.139535,10.697674,3.0,0.0
1611,Jeremie Frimpong,NED,22.0,25.0,61.710000,0.36,0.28,0.33,0.33,0.23,6.120000,1.960000,13.920000,6.0,0.0
379,Andrew Robertson,SCO,29.0,18.8,52.198298,0.16,0.11,0.10,0.10,0.29,3.351064,7.446809,6.914894,2.0,0.0
1640,Alphonso Davies,CAN,22.0,22.9,50.198646,0.09,0.22,0.04,0.04,0.12,5.283843,5.109170,6.069869,0.0,0.0
1609,Álex Grimaldo,ESP,27.0,30.9,49.483689,0.32,0.42,0.18,0.18,0.30,2.491909,5.242718,8.446602,1.0,0.0
2971,Alejandro Balde,ESP,19.0,15.3,49.310000,0.00,0.07,0.01,0.01,0.08,4.771242,2.679739,9.215686,1.0,0.0
3377,Lucas Vázquez,ESP,32.0,15.8,48.208861,0.19,0.38,0.07,0.07,0.24,1.772152,3.670886,10.126582,0.0,0.0
3376,Fran Garcia,ESP,23.0,15.5,47.824194,0.06,0.32,0.03,0.03,0.16,3.419355,3.870968,8.967742,2.0,0.0
1648,Noussair Mazraoui,MAR,25.0,13.3,46.511805,0.00,0.23,0.07,0.07,0.13,2.556391,7.744361,5.939850,3.0,0.0
2410,Jonathan Clauss,FRA,30.0,23.0,45.930000,0.13,0.17,0.06,0.06,0.20,2.565217,5.956522,7.478261,2.0,1.0



Top 10 GK (eligible; n=81):


,Player,Nation_code,Age,Playing Time 90s,weighted_score_norm,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,Performance CrdY,Performance CrdR
157,Jason Steele,ENG,32.0,17.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0
2368,Anthony Lopes,POR,32.0,31.0,0.0,0.0,0.0,0.0,0.0,0.01,0.032258,0.032258,0.0,0.0,0.0
2488,Philipp Köhn,SUI,25.0,22.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0
1998,Janis Blaswich,GER,32.0,21.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.142857,0.0,0.0,0.0
1896,Marvin Schwäbe,GER,28.0,34.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.088235,0.0,0.0,0.0
1807,Noah Atubolu,GER,21.0,34.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.029412,0.0,0.0,0.0
1404,Guillermo Ochoa,MEX,38.0,21.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0
2977,Iñaki Peña,ESP,24.0,10.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0
2996,Jeremías Ledesma,ARG,30.0,33.9,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0
1411,Benoît Costil,FRA,36.0,13.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0


In [36]:
# Create an improved normalized score: progression per-90 AND cards per-90 (instead of season totals)
import numpy as np

scoring_norm2_df = filtered_players_stats.copy()

# Ensure numeric columns used
cols_needed2 = [
    feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], feature_cols["xag_p90"],
    feature_cols["prgc"], feature_cols["prgp"], feature_cols["prgr"],
    feature_cols["crdy"], feature_cols["crdr"],
    "Per 90 Minutes npxG", "Playing Time 90s",
]
for c in cols_needed2:
    scoring_norm2_df[c] = pd.to_numeric(scoring_norm2_df[c], errors="coerce")

# Avoid divide-by-zero
n90 = scoring_norm2_df["Playing Time 90s"].replace({0: np.nan})

# Per-90 rate conversions for totals
scoring_norm2_df["PrgC_p90"] = scoring_norm2_df[feature_cols["prgc"]] / n90
scoring_norm2_df["PrgP_p90"] = scoring_norm2_df[feature_cols["prgp"]] / n90
scoring_norm2_df["PrgR_p90"] = scoring_norm2_df[feature_cols["prgr"]] / n90
scoring_norm2_df["CrdY_p90"] = scoring_norm2_df[feature_cols["crdy"]] / n90
scoring_norm2_df["CrdR_p90"] = scoring_norm2_df[feature_cols["crdr"]] / n90

# xG/npxG blend per 90
scoring_norm2_df["xg_npxg_p90_blend"] = scoring_norm2_df[[feature_cols["xg_p90"], "Per 90 Minutes npxG"]].mean(axis=1)


def compute_weighted_score_norm2(row):
    pos = row.get("Pos_group")
    if pd.isna(pos) or pos not in weights_by_pos:
        return np.nan
    w = weights_by_pos[pos]

    # positive contributions
    score = (
        w["gls_p90"] * row[feature_cols["gls_p90"]] +
        w["ast_p90"] * row[feature_cols["ast_p90"]] +
        w["xg_p90"] * row["xg_npxg_p90_blend"] +
        w["xag_p90"] * row[feature_cols["xag_p90"]] +
        w["prgc"] * row["PrgC_p90"] +
        w["prgp"] * row["PrgP_p90"] +
        w["prgr"] * row["PrgR_p90"]
    )

    # subtract card rates (per 90)
    score = score - (w["crdy"] * row["CrdY_p90"] + w["crdr"] * row["CrdR_p90"])
    return score

scoring_norm2_df["weighted_score_norm2"] = scoring_norm2_df.apply(compute_weighted_score_norm2, axis=1)

print("weighted_score_norm2 missing rate:", scoring_norm2_df["weighted_score_norm2"].isna().mean())
display(scoring_norm2_df["weighted_score_norm2"].describe())

# Create an eligibility cut to stabilize per-90 rates
min_90s_norm2 = 10
eligible2 = scoring_norm2_df[scoring_norm2_df["Playing Time 90s"] >= min_90s_norm2].copy()

print(f"\nEligible2 players with Playing Time 90s >= {min_90s_norm2}: {len(eligible2):,} / {len(scoring_norm2_df):,}")
print("\nTop 10 overall (eligible2):")
display(
    eligible2[["Player","Pos_group","Nation_code","Age","Playing Time 90s","weighted_score_norm2"]]
    .sort_values("weighted_score_norm2", ascending=False)
    .head(10)
)

for g in ["FWD","MID","DEF"]:
    subg2 = eligible2[eligible2["Pos_group"] == g].sort_values("weighted_score_norm2", ascending=False)
    print(f"\nTop 10 {g} (eligible2; n={len(subg2)}):")
    display(subg2[[
        "Player","Nation_code","Age","Playing Time 90s","weighted_score_norm2",
        feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], "Per 90 Minutes npxG",
        feature_cols["xag_p90"], "PrgC_p90","PrgP_p90","PrgR_p90","CrdY_p90","CrdR_p90",
    ]].head(10))

# GK is intentionally near-zero under these attacking/progression features
print("\nGK note: under the requested feature set, GK scores will be ~0 (no GK-relevant features included).")


weighted_score_norm2 missing rate: 0.1951987406532861


count    2045.000000
mean       24.876164
std        16.854353
min       -10.000000
25%        15.303043
50%        23.445806
75%        32.653439
max       257.870000
Name: weighted_score_norm2, dtype: float64


Eligible2 players with Playing Time 90s >= 10: 1,201 / 2,541

Top 10 overall (eligible2):


,Player,Pos_group,Nation_code,Age,Playing Time 90s,weighted_score_norm2
1612,Florian Wirtz,MID,GER,20.0,26.4,79.415455
2640,Achraf Hakimi,DEF,MAR,24.0,21.5,75.423023
459,Kevin De Bruyne,MID,BEL,32.0,13.6,69.754118
2645,Ousmane Dembélé,FWD,FRA,26.0,16.7,67.750659
1611,Jeremie Frimpong,DEF,NED,22.0,25.0,67.470000
457,Jeremy Doku,FWD,BEL,21.0,17.7,66.587288
1649,Kingsley Coman,FWD,FRA,27.0,12.4,59.980000
1992,Xavi Simons,MID,NED,20.0,29.5,59.082203
2,Martin Ødegaard,MID,NOR,24.0,34.3,58.215977
647,James Maddison,MID,ENG,26.0,23.7,58.002068



Top 10 FWD (eligible2; n=375):


...


Top 10 MID (eligible2; n=332):


,Player,Nation_code,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
1612,Florian Wirtz,GER,20.0,26.4,79.415455,0.42,0.42,0.34,0.31,0.29,5.227273,8.484848,11.590909,0.113636,0.000000
459,Kevin De Bruyne,BEL,32.0,13.6,69.754118,0.29,0.74,0.18,0.18,0.85,3.455882,8.161765,9.779412,0.147059,0.000000
1992,Xavi Simons,NED,20.0,29.5,59.082203,0.27,0.37,0.28,0.23,0.33,5.118644,6.338983,7.322034,0.372881,0.033898
2,Martin Ødegaard,NOR,24.0,34.3,58.215977,0.23,0.29,0.22,0.17,0.28,2.653061,10.029155,5.889213,0.058309,0.000000
647,James Maddison,ENG,26.0,23.7,58.002068,0.17,0.38,0.27,0.27,0.25,2.447257,9.451477,6.582278,0.210970,0.000000
2927,Samuel Lino,BRA,23.0,23.1,57.910866,0.17,0.22,0.24,0.24,0.21,5.670996,3.636364,9.307359,0.043290,0.000000
3364,Jude Bellingham,ENG,20.0,25.7,56.859572,0.74,0.23,0.43,0.40,0.21,3.307393,7.626459,6.887160,0.194553,0.038911
2933,Rodrigo Riquelme,ESP,23.0,17.0,56.031176,0.18,0.29,0.16,0.16,0.18,5.117647,3.117647,9.764706,0.058824,0.000000
3324,Isco,ESP,31.0,25.8,55.737054,0.31,0.19,0.32,0.26,0.25,3.875969,8.372093,5.620155,0.387597,0.000000
3373,Luka Modrić,CRO,37.0,18.8,55.411489,0.11,0.32,0.07,0.07,0.30,2.819149,10.425532,4.521277,0.106383,0.000000



Top 10 DEF (eligible2; n=413):


,Player,Nation_code,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
2640,Achraf Hakimi,MAR,24.0,21.5,75.423023,0.19,0.23,0.20,0.20,0.24,3.906977,10.139535,10.697674,0.139535,0.000000
1611,Jeremie Frimpong,NED,22.0,25.0,67.470000,0.36,0.28,0.33,0.33,0.23,6.120000,1.960000,13.920000,0.240000,0.000000
379,Andrew Robertson,SCO,29.0,18.8,54.091915,0.16,0.11,0.10,0.10,0.29,3.351064,7.446809,6.914894,0.106383,0.000000
1609,Álex Grimaldo,ESP,27.0,30.9,50.451327,0.32,0.42,0.18,0.18,0.30,2.491909,5.242718,8.446602,0.032362,0.000000
2971,Alejandro Balde,ESP,19.0,15.3,50.244641,0.00,0.07,0.01,0.01,0.08,4.771242,2.679739,9.215686,0.065359,0.000000
1640,Alphonso Davies,CAN,22.0,22.9,50.198646,0.09,0.22,0.04,0.04,0.12,5.283843,5.109170,6.069869,0.000000,0.000000
3376,Fran Garcia,ESP,23.0,15.5,49.695161,0.06,0.32,0.03,0.03,0.16,3.419355,3.870968,8.967742,0.129032,0.000000
1648,Noussair Mazraoui,MAR,25.0,13.3,49.286241,0.00,0.23,0.07,0.07,0.13,2.556391,7.744361,5.939850,0.225564,0.000000
2410,Jonathan Clauss,FRA,30.0,23.0,48.799565,0.13,0.17,0.06,0.06,0.20,2.565217,5.956522,7.478261,0.086957,0.043478
162,Pervis Estupiñán,ECU,25.0,13.8,48.251449,0.14,0.22,0.07,0.07,0.25,3.405797,6.014493,6.376812,0.289855,0.000000



GK note: under the requested feature set, GK scores will be ~0 (no GK-relevant features included).


In [37]:
# Create a final, tidy ranking table from the normalized per-90 score (weighted_score_norm2)
# - Uses eligible2 (Playing Time 90s >= 10)
# - Provides overall and per-position top lists

final_rankings = eligible2.copy()

# Columns to keep for downstream use
keep_cols = [
    "Player","Nation","Nation_code","Pos","Pos_group","Age","Playing Time 90s",
    "weighted_score_norm2",
    feature_cols["gls_p90"], feature_cols["ast_p90"], feature_cols["xg_p90"], "Per 90 Minutes npxG",
    feature_cols["xag_p90"],
    "PrgC_p90","PrgP_p90","PrgR_p90","CrdY_p90","CrdR_p90",
]
keep_cols = [c for c in keep_cols if c in final_rankings.columns]
final_rankings = final_rankings[keep_cols].sort_values("weighted_score_norm2", ascending=False)

print("final_rankings shape:", final_rankings.shape)
print("Score column: weighted_score_norm2 (higher is better)")

print("\nTop 15 overall:")
display(final_rankings.head(15))

for g in ["FWD","MID","DEF","GK"]:
    sub = final_rankings[final_rankings["Pos_group"] == g]
    print(f"\nTop 15 {g} (n={len(sub)}):")
    display(sub.head(15))

# Optional: save to CSV for user download/use
out_path = "final_rankings_weighted_score_norm2.csv"
final_rankings.to_csv(out_path, index=False)
print(f"\nSaved: {out_path}")


final_rankings shape: (1201, 18)
Score column: weighted_score_norm2 (higher is better)

Top 15 overall:


,Player,Nation,Nation_code,Pos,Pos_group,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
1612,Florian Wirtz,de GER,GER,MF,MID,20.0,26.4,79.415455,0.42,0.42,0.34,0.31,0.29,5.227273,8.484848,11.590909,0.113636,0.000000
2640,Achraf Hakimi,ma MAR,MAR,DF,DEF,24.0,21.5,75.423023,0.19,0.23,0.20,0.20,0.24,3.906977,10.139535,10.697674,0.139535,0.000000
459,Kevin De Bruyne,be BEL,BEL,MF,MID,32.0,13.6,69.754118,0.29,0.74,0.18,0.18,0.85,3.455882,8.161765,9.779412,0.147059,0.000000
2645,Ousmane Dembélé,fr FRA,FRA,"MF,FW",FWD,26.0,16.7,67.750659,0.18,0.48,0.26,0.26,0.40,9.580838,7.964072,14.790419,0.000000,0.000000
1611,Jeremie Frimpong,nl NED,NED,DF,DEF,22.0,25.0,67.470000,0.36,0.28,0.33,0.33,0.23,6.120000,1.960000,13.920000,0.240000,0.000000
457,Jeremy Doku,be BEL,BEL,"FW,MF",FWD,21.0,17.7,66.587288,0.17,0.45,0.14,0.14,0.26,12.316384,3.220339,16.666667,0.169492,0.000000
1649,Kingsley Coman,fr FRA,FRA,FW,FWD,27.0,12.4,59.980000,0.24,0.24,0.22,0.22,0.31,6.048387,5.806452,16.935484,0.080645,0.000000
1992,Xavi Simons,nl NED,NED,MF,MID,20.0,29.5,59.082203,0.27,0.37,0.28,0.23,0.33,5.118644,6.338983,7.322034,0.372881,0.033898
2,Martin Ødegaard,no NOR,NOR,MF,MID,24.0,34.3,58.215977,0.23,0.29,0.22,0.17,0.28,2.653061,10.029155,5.889213,0.058309,0.000000
647,James Maddison,eng ENG,ENG,MF,MID,26.0,23.7,58.002068,0.17,0.38,0.27,0.27,0.25,2.447257,9.451477,6.582278,0.210970,0.000000



Top 15 FWD (n=375):


,Player,Nation,Nation_code,Pos,Pos_group,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
2645,Ousmane Dembélé,fr FRA,FRA,"MF,FW",FWD,26.0,16.7,67.750659,0.18,0.48,0.26,0.26,0.40,9.580838,7.964072,14.790419,0.000000,0.000000
457,Jeremy Doku,be BEL,BEL,"FW,MF",FWD,21.0,17.7,66.587288,0.17,0.45,0.14,0.14,0.26,12.316384,3.220339,16.666667,0.169492,0.000000
1649,Kingsley Coman,fr FRA,FRA,FW,FWD,27.0,12.4,59.980000,0.24,0.24,0.22,0.22,0.31,6.048387,5.806452,16.935484,0.080645,0.000000
159,Kaoru Mitoma,jp JPN,JPN,FW,FWD,26.0,16.5,53.121515,0.18,0.24,0.21,0.21,0.16,7.333333,3.878788,14.484848,0.242424,0.000000
4,Bukayo Saka,eng ENG,ENG,FW,FWD,21.0,32.4,52.465247,0.49,0.28,0.48,0.33,0.32,4.783951,3.888889,15.679012,0.123457,0.000000
461,Jack Grealish,eng ENG,ENG,"FW,MF",FWD,27.0,11.2,52.000000,0.27,0.09,0.19,0.19,0.22,7.767857,4.464286,13.035714,0.535714,0.000000
2643,Bradley Barcola,fr FRA,FRA,"FW,MF",FWD,20.0,15.2,51.043158,0.26,0.46,0.30,0.30,0.34,6.118421,3.684211,14.210526,0.197368,0.065789
8,Gabriel Martinelli,br BRA,BRA,FW,FWD,22.0,22.4,50.541786,0.27,0.18,0.31,0.31,0.27,5.669643,2.901786,15.401786,0.044643,0.000000
3368,Vinicius Júnior,br BRA,BRA,FW,FWD,23.0,20.7,50.059082,0.72,0.24,0.63,0.59,0.21,6.618357,2.753623,13.381643,0.338164,0.000000
2634,Kylian Mbappé,fr FRA,FRA,FW,FWD,24.0,24.0,49.988333,1.13,0.29,0.87,0.60,0.24,4.750000,5.083333,11.916667,0.166667,0.000000



Top 15 MID (n=332):


...


Top 15 DEF (n=413):


,Player,Nation,Nation_code,Pos,Pos_group,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
2640,Achraf Hakimi,ma MAR,MAR,DF,DEF,24.0,21.5,75.423023,0.19,0.23,0.20,0.20,0.24,3.906977,10.139535,10.697674,0.139535,0.000000
1611,Jeremie Frimpong,nl NED,NED,DF,DEF,22.0,25.0,67.470000,0.36,0.28,0.33,0.33,0.23,6.120000,1.960000,13.920000,0.240000,0.000000
379,Andrew Robertson,sct SCO,SCO,DF,DEF,29.0,18.8,54.091915,0.16,0.11,0.10,0.10,0.29,3.351064,7.446809,6.914894,0.106383,0.000000
1609,Álex Grimaldo,es ESP,ESP,DF,DEF,27.0,30.9,50.451327,0.32,0.42,0.18,0.18,0.30,2.491909,5.242718,8.446602,0.032362,0.000000
2971,Alejandro Balde,es ESP,ESP,DF,DEF,19.0,15.3,50.244641,0.00,0.07,0.01,0.01,0.08,4.771242,2.679739,9.215686,0.065359,0.000000
1640,Alphonso Davies,ca CAN,CAN,DF,DEF,22.0,22.9,50.198646,0.09,0.22,0.04,0.04,0.12,5.283843,5.109170,6.069869,0.000000,0.000000
3376,Fran Garcia,es ESP,ESP,DF,DEF,23.0,15.5,49.695161,0.06,0.32,0.03,0.03,0.16,3.419355,3.870968,8.967742,0.129032,0.000000
1648,Noussair Mazraoui,ma MAR,MAR,DF,DEF,25.0,13.3,49.286241,0.00,0.23,0.07,0.07,0.13,2.556391,7.744361,5.939850,0.225564,0.000000
2410,Jonathan Clauss,fr FRA,FRA,DF,DEF,30.0,23.0,48.799565,0.13,0.17,0.06,0.06,0.20,2.565217,5.956522,7.478261,0.086957,0.043478
162,Pervis Estupiñán,ec ECU,ECU,DF,DEF,25.0,13.8,48.251449,0.14,0.22,0.07,0.07,0.25,3.405797,6.014493,6.376812,0.289855,0.000000



Top 15 GK (n=81):


,Player,Nation,Nation_code,Pos,Pos_group,Age,Playing Time 90s,weighted_score_norm2,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes xG,Per 90 Minutes npxG,Per 90 Minutes xAG,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
1998,Janis Blaswich,de GER,GER,GK,GK,32.0,21.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.142857,0.0,0.000000,0.0
1404,Guillermo Ochoa,mx MEX,MEX,GK,GK,38.0,21.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
1411,Benoît Costil,fr FRA,FRA,GK,GK,36.0,13.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
2368,Anthony Lopes,pt POR,POR,GK,GK,32.0,31.0,0.000000,0.0,0.00,0.0,0.0,0.01,0.032258,0.032258,0.0,0.000000,0.0
2977,Iñaki Peña,es ESP,ESP,GK,GK,24.0,10.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
1807,Noah Atubolu,de GER,GER,GK,GK,21.0,34.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.029412,0.0,0.000000,0.0
2996,Jeremías Ledesma,ar ARG,ARG,GK,GK,30.0,33.9,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
2488,Philipp Köhn,ch SUI,SUI,GK,GK,25.0,22.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
800,Juan Musso,ar ARG,ARG,GK,GK,29.0,11.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.0
1896,Marvin Schwäbe,de GER,GER,GK,GK,28.0,34.0,0.000000,0.0,0.00,0.0,0.0,0.00,0.000000,0.088235,0.0,0.000000,0.0



Saved: final_rankings_weighted_score_norm2.csv


In [38]:
# Validate score drivers and create a robust (winsorized + standardized) alternative score by position
import numpy as np
import pandas as pd

# Work from eligible2 (already has per-90 rates + weighted_score_norm2)
robust_df = eligible2.copy()

# Columns that feed the score (per-90 versions)
robust_components = {
    "gls_p90": feature_cols["gls_p90"],
    "ast_p90": feature_cols["ast_p90"],
    "xg_p90": feature_cols["xg_p90"],
    "npxg_p90": "Per 90 Minutes npxG",
    "xag_p90": feature_cols["xag_p90"],
    "prgc_p90": "PrgC_p90",
    "prgp_p90": "PrgP_p90",
    "prgr_p90": "PrgR_p90",
    "crdy_p90": "CrdY_p90",
    "crdr_p90": "CrdR_p90",
}

# Ensure numeric
to_num = list(robust_components.values()) + ["Playing Time 90s", "weighted_score_norm2"]
for c in to_num:
    if c in robust_df.columns:
        robust_df[c] = pd.to_numeric(robust_df[c], errors="coerce")

# xG/npxG blend per-90 (same as earlier)
robust_df["xg_npxg_p90_blend"] = robust_df[[robust_components["xg_p90"], robust_components["npxg_p90"]]].mean(axis=1)

# Quick distribution check (helps spot outliers)
check_cols = [
    "weighted_score_norm2",
    robust_components["prgc_p90"], robust_components["prgp_p90"], robust_components["prgr_p90"],
    robust_components["crdy_p90"], robust_components["crdr_p90"],
]
print("Distribution snapshot (eligible2):")
display(robust_df[check_cols].describe(percentiles=[0.01,0.05,0.5,0.95,0.99]))

# Build a robust score: winsorize each component within position group, then z-score within group
# (keeps interpretability: score is still a weighted sum, but components are on comparable scales)

def winsorize_series(s, lo=0.01, hi=0.99):
    qlo, qhi = s.quantile([lo, hi])
    return s.clip(lower=qlo, upper=qhi)

def zscore(s):
    mu = s.mean()
    sd = s.std(ddof=0)
    if sd == 0 or np.isnan(sd):
        return s * 0
    return (s - mu) / sd

robust_score_parts = []

for pos in ["FWD", "MID", "DEF", "GK"]:
    mask = robust_df["Pos_group"] == pos
    if mask.sum() == 0:
        continue

    tmp = robust_df.loc[mask, :].copy()

    # winsorize + zscore components
    for k, col in robust_components.items():
        if col not in tmp.columns:
            continue
        tmp[col + "_w"] = winsorize_series(tmp[col])
        tmp[col + "_z"] = zscore(tmp[col + "_w"])

    tmp["xg_npxg_p90_blend_w"] = winsorize_series(tmp["xg_npxg_p90_blend"])
    tmp["xg_npxg_p90_blend_z"] = zscore(tmp["xg_npxg_p90_blend_w"])

    w = weights_by_pos[pos]

    # Weighted sum on standardized components; cards subtract
    tmp["weighted_score_robust_z"] = (
        w["gls_p90"] * tmp[robust_components["gls_p90"] + "_z"] +
        w["ast_p90"] * tmp[robust_components["ast_p90"] + "_z"] +
        w["xg_p90"]  * tmp["xg_npxg_p90_blend_z"] +
        w["xag_p90"] * tmp[robust_components["xag_p90"] + "_z"] +
        w["prgc"]    * tmp[robust_components["prgc_p90"] + "_z"] +
        w["prgp"]    * tmp[robust_components["prgp_p90"] + "_z"] +
        w["prgr"]    * tmp[robust_components["prgr_p90"] + "_z"] -
        w["crdy"]    * tmp[robust_components["crdy_p90"] + "_z"] -
        w["crdr"]    * tmp[robust_components["crdr_p90"] + "_z"]
    )

    robust_score_parts.append(tmp[["Player","Nation_code","Age","Pos","Pos_group","Playing Time 90s","weighted_score_norm2","weighted_score_robust_z"]])

robust_rankings = pd.concat(robust_score_parts, axis=0).sort_values("weighted_score_robust_z", ascending=False)

print("\nTop 15 overall by robust z-score:")
display(robust_rankings.head(15))

for pos in ["FWD","MID","DEF","GK"]:
    subpos = robust_rankings[robust_rankings["Pos_group"] == pos].sort_values("weighted_score_robust_z", ascending=False)
    print(f"\nTop 10 {pos} by robust z-score (n={len(subpos)}):")
    display(subpos.head(10))

# Save for comparison
robust_rankings.to_csv("final_rankings_weighted_score_robust_z.csv", index=False)
print("\nSaved: final_rankings_weighted_score_robust_z.csv")


Distribution snapshot (eligible2):


,weighted_score_norm2,PrgC_p90,PrgP_p90,PrgR_p90,CrdY_p90,CrdR_p90
count,1201.000000,1201.000000,1201.000000,1201.000000,1201.000000,1201.000000
mean,23.940936,1.694752,3.683815,3.629653,0.199552,0.009151
std,12.531259,1.366279,1.961926,3.201502,0.128367,0.023400
min,-0.176991,0.000000,0.000000,0.000000,0.000000,0.000000
1%,-0.119403,0.000000,0.000000,0.000000,0.000000,0.000000
5%,-0.030303,0.000000,0.060606,0.000000,0.000000,0.000000
50%,23.332727,1.417323,3.578595,3.140097,0.181818,0.000000
95%,45.322239,4.290541,7.225131,9.436620,0.428571,0.061728
99%,56.031176,5.833333,8.744589,12.092199,0.548523,0.099010
max,79.415455,12.316384,12.523962,16.935484,0.904255,0.198020



Top 15 overall by robust z-score:


,Player,Nation_code,Age,Pos,Pos_group,Playing Time 90s,weighted_score_norm2,weighted_score_robust_z
1612,Florian Wirtz,GER,20.0,MF,MID,26.4,79.415455,58.592443
459,Kevin De Bruyne,BEL,32.0,MF,MID,13.6,69.754118,48.843874
1992,Xavi Simons,NED,20.0,MF,MID,29.5,59.082203,45.986707
2640,Achraf Hakimi,MAR,24.0,DF,DEF,21.5,75.423023,43.824345
647,James Maddison,ENG,26.0,MF,MID,23.7,58.002068,39.895843
1609,Álex Grimaldo,ESP,27.0,DF,DEF,30.9,50.451327,39.644403
2,Martin Ødegaard,NOR,24.0,MF,MID,34.3,58.215977,39.358847
2002,Dani Olmo,ESP,25.0,MF,MID,16.1,47.404161,38.878179
1674,Kevin Stöger,AUT,29.0,MF,MID,29.6,53.509459,38.142786
3364,Jude Bellingham,ENG,20.0,MF,MID,25.7,56.859572,37.857758



Top 10 FWD by robust z-score (n=375):


...


Top 10 DEF by robust z-score (n=413):


,Player,Nation_code,Age,Pos,Pos_group,Playing Time 90s,weighted_score_norm2,weighted_score_robust_z
2640,Achraf Hakimi,MAR,24.0,DF,DEF,21.5,75.423023,43.824345
1609,Álex Grimaldo,ESP,27.0,DF,DEF,30.9,50.451327,39.644403
379,Andrew Robertson,SCO,29.0,DF,DEF,18.8,54.091915,35.368991
1611,Jeremie Frimpong,NED,22.0,DF,DEF,25.0,67.470000,33.335607
162,Pervis Estupiñán,ECU,25.0,DF,DEF,13.8,48.251449,30.658264
3377,Lucas Vázquez,ESP,32.0,DF,DEF,15.8,48.208861,29.040294
3376,Fran Garcia,ESP,23.0,DF,DEF,15.5,49.695161,27.865931
375,Trent Alexander-Arnold,ENG,24.0,DF,DEF,23.9,44.480439,26.902973
1640,Alphonso Davies,CAN,22.0,DF,DEF,22.9,50.198646,26.704580
1112,Denzel Dumfries,NED,27.0,DF,DEF,18.2,42.034286,26.516800



Top 10 GK by robust z-score (n=81):


,Player,Nation_code,Age,Pos,Pos_group,Playing Time 90s,weighted_score_norm2,weighted_score_robust_z
2488,Philipp Köhn,SUI,25.0,GK,GK,22.0,0.0,1.73174
2977,Iñaki Peña,ESP,24.0,GK,GK,10.0,0.0,1.73174
2996,Jeremías Ledesma,ARG,30.0,GK,GK,33.9,0.0,1.73174
1896,Marvin Schwäbe,GER,28.0,GK,GK,34.0,0.0,1.73174
1411,Benoît Costil,FRA,36.0,GK,GK,13.0,0.0,1.73174
157,Jason Steele,ENG,32.0,GK,GK,17.0,0.0,1.73174
1998,Janis Blaswich,GER,32.0,GK,GK,21.0,0.0,1.73174
800,Juan Musso,ARG,29.0,GK,GK,11.0,0.0,1.73174
1807,Noah Atubolu,GER,21.0,GK,GK,34.0,0.0,1.73174
2368,Anthony Lopes,POR,32.0,GK,GK,31.0,0.0,1.73174



Saved: final_rankings_weighted_score_robust_z.csv


In [39]:
# Create nation-level rankings from player-level final_rankings
import pandas as pd
import numpy as np

# Choose which player-level score to rank nations by
# - 'weighted_score_norm2' is the main per-90 normalized score
# - 'weighted_score_robust_z' is the outlier-resistant alternative (position-standardized)
score_col = "weighted_score_norm2"
alt_score_col = "weighted_score_robust_z" if "weighted_score_robust_z" in robust_rankings.columns else None

# Base table: use final_rankings (already filtered to Playing Time 90s >= 10 and allowed nations)
base = final_rankings.copy()

# Nation summary stats
nation_rankings = (
    base.groupby("Nation_code", as_index=False)
        .agg(
            n_players=("Player", "count"),
            minutes90_sum=("Playing Time 90s", "sum"),
            score_mean=(score_col, "mean"),
            score_median=(score_col, "median"),
            score_top1=(score_col, "max"),
        )
)

# Add a minutes-weighted average score (weights = Playing Time 90s)
def wavg(g, value_col, weight_col):
    v = g[value_col].to_numpy()
    w = g[weight_col].to_numpy()
    m = np.isfinite(v) & np.isfinite(w)
    v = v[m]; w = w[m]
    if w.sum() == 0:
        return np.nan
    return np.sum(v * w) / np.sum(w)

wavg_scores = (
    base.groupby("Nation_code")
        .apply(lambda g: wavg(g, score_col, "Playing Time 90s"), include_groups=False)
        .rename("score_wavg_by_minutes")
        .reset_index()
)

nation_rankings = nation_rankings.merge(wavg_scores, on="Nation_code", how="left")

# Attach best player name per nation for interpretability
idx_best = base.groupby("Nation_code")[score_col].idxmax()
best_players = (
    base.loc[idx_best, ["Nation_code", "Player", "Pos_group", "Playing Time 90s", score_col]]
        .rename(columns={
            "Player": "best_player",
            "Pos_group": "best_player_pos_group",
            "Playing Time 90s": "best_player_90s",
            score_col: "best_player_score",
        })
)

nation_rankings = nation_rankings.merge(best_players, on="Nation_code", how="left")

# Optional: also compute nation aggregates for the robust z-score
if alt_score_col is not None:
    rb = robust_rankings[["Nation_code", "Player", "Playing Time 90s", alt_score_col]].copy()
    rb = rb.merge(base[["Nation_code","Player","Playing Time 90s"]], on=["Nation_code","Player","Playing Time 90s"], how="inner")
    rb_summary = (
        rb.groupby("Nation_code", as_index=False)
          .agg(robust_z_mean=(alt_score_col, "mean"), robust_z_median=(alt_score_col, "median"))
    )
    nation_rankings = nation_rankings.merge(rb_summary, on="Nation_code", how="left")

# Sort nations by minutes-weighted average score as the default nation ranking
nation_rankings = nation_rankings.sort_values(["score_wavg_by_minutes", "score_mean"], ascending=False)

print("Nation rankings table shape:", nation_rankings.shape)
print("Ranking metric (primary): score_wavg_by_minutes using", score_col)

display(nation_rankings.head(25))

# Save
nation_out_path = "nation_rankings_from_weighted_score_norm2.csv"
nation_rankings.to_csv(nation_out_path, index=False)
print("Saved:", nation_out_path)


Nation rankings table shape: (37, 13)
Ranking metric (primary): score_wavg_by_minutes using weighted_score_norm2


,Nation_code,n_players,minutes90_sum,score_mean,score_median,score_top1,score_wavg_by_minutes,best_player,best_player_pos_group,best_player_90s,best_player_score,robust_z_mean,robust_z_median
19,JPN,12,248.1,32.590009,33.545392,53.121515,32.852503,Kaoru Mitoma,FWD,16.5,53.121515,4.988605,7.617589
11,EGY,3,76.0,30.489518,27.664687,46.919362,31.660237,Mohamed Salah,FWD,28.2,46.919362,10.114230,6.698395
5,CAN,3,72.4,30.504456,26.169795,50.198646,30.678854,Alphonso Davies,DEF,22.9,50.198646,8.436838,8.590495
18,JOR,1,21.4,30.017757,30.017757,30.017757,30.017757,Musa Al-Taamari,FWD,21.4,30.017757,-0.694731,-0.694731
30,SCO,13,252.5,28.977434,29.242415,54.091915,29.318535,Andrew Robertson,DEF,18.8,54.091915,2.472814,0.449455
20,KOR,6,141.3,26.375495,20.732961,39.671196,26.674593,Son Heung-min,FWD,32.6,39.671196,0.936819,-2.128983
2,AUT,24,527.4,26.095264,25.504292,53.509459,26.655283,Kevin Stöger,MID,29.6,53.509459,1.219418,-0.020670
26,PAN,1,11.1,25.784865,25.784865,25.784865,25.784865,Michael Amir Murillo,DEF,11.1,25.784865,8.904624,8.904624
10,ECU,4,95.1,29.993415,27.918076,48.251449,25.686562,Pervis Estupiñán,DEF,13.8,48.251449,7.737404,4.935528
9,CRO,20,435.7,26.186983,25.231767,55.411489,25.389885,Luka Modrić,MID,18.8,55.411489,1.254012,-1.341201


Saved: nation_rankings_from_weighted_score_norm2.csv


In [40]:
# Sort nation_rankings from best-player score to worst-player score

nation_rankings_best_player_sorted = nation_rankings.sort_values(
    ["best_player_score", "score_wavg_by_minutes", "score_mean"],
    ascending=[False, False, False],
).reset_index(drop=True)

print("Nation rankings sorted by best_player_score (desc).")
display(nation_rankings_best_player_sorted.head(50))

# Save
out_path_best_player = "nation_rankings_sorted_by_best_player_score.csv"
nation_rankings_best_player_sorted.to_csv(out_path_best_player, index=False)
print("Saved:", out_path_best_player)


Nation rankings sorted by best_player_score (desc).


,Nation_code,n_players,minutes90_sum,score_mean,score_median,score_top1,score_wavg_by_minutes,best_player,best_player_pos_group,best_player_90s,best_player_score,robust_z_mean,robust_z_median
0,GER,157,3363.9,22.311880,22.463333,79.415455,21.529239,Florian Wirtz,MID,26.4,79.415455,0.210149,-0.885099
1,MAR,21,405.6,26.736615,23.186923,75.423023,24.891166,Achraf Hakimi,DEF,21.5,75.423023,-0.001416,-1.827657
2,BEL,31,631.4,22.937811,23.617895,69.754118,21.700343,Kevin De Bruyne,MID,13.6,69.754118,-0.165781,-0.687320
3,FRA,187,3922.2,24.968195,24.850303,67.750659,24.199647,Ousmane Dembélé,FWD,16.7,67.750659,-0.153839,-0.881603
4,NED,43,965.8,24.787015,22.425772,67.470000,24.024995,Jeremie Frimpong,DEF,25.0,67.470000,2.444371,-0.908283
5,NOR,15,337.0,22.693005,22.295632,58.215977,24.334481,Martin Ødegaard,MID,34.3,58.215977,1.977796,-0.174757
6,ENG,113,2535.6,23.299600,22.278636,58.002068,23.022707,James Maddison,MID,23.7,58.002068,0.309521,-1.167643
7,BRA,70,1586.0,24.526568,24.138195,57.910866,23.847986,Samuel Lino,MID,23.1,57.910866,-0.544659,-1.067300
8,ESP,226,4889.8,24.056718,24.454116,56.031176,23.360857,Rodrigo Riquelme,MID,17.0,56.031176,-0.371989,-0.985092
9,CRO,20,435.7,26.186983,25.231767,55.411489,25.389885,Luka Modrić,MID,18.8,55.411489,1.254012,-1.341201


Saved: nation_rankings_sorted_by_best_player_score.csv


In [44]:
# Compute nation "Starting XI Strength" using top 11 players by weighted_score_norm2
import numpy as np
import pandas as pd

score_col = "weighted_score_norm2"
assert score_col in final_rankings.columns, f"Missing {score_col} in final_rankings"

base_xi = final_rankings.copy()

# Rank players within nation by score (descending)
base_xi["rank_in_nation"] = (
    base_xi.groupby("Nation_code")[score_col]
           .rank(method="first", ascending=False)
)

# Keep top 11 (or fewer if nation has <11 eligible players)
xi = base_xi[base_xi["rank_in_nation"] <= 11].copy()

# Aggregate XI strength metrics
nation_xi = (
    xi.groupby("Nation_code", as_index=False)
      .agg(
          n_xi_players=("Player", "count"),
          xi_score_sum=(score_col, "sum"),
          xi_score_mean=(score_col, "mean"),
          xi_score_median=(score_col, "median"),
          xi_score_min=(score_col, "min"),
          xi_score_max=(score_col, "max"),
          xi_minutes90_sum=("Playing Time 90s", "sum"),
      )
)

# Attach the XI list for interpretability
xi_lists = (
    xi.sort_values(["Nation_code", score_col], ascending=[True, False])
      .groupby("Nation_code")
      .apply(lambda g: ", ".join(g["Player"].head(11).tolist()), include_groups=False)
      .rename("starting_xi_players")
      .reset_index()
)

nation_xi = nation_xi.merge(xi_lists, on="Nation_code", how="left")

# Sort nations by XI strength (use mean by default; sum is highly correlated but depends on n_xi_players)
nation_xi = nation_xi.sort_values(["xi_score_mean", "xi_score_sum"], ascending=False).reset_index(drop=True)

print("Nation Starting XI Strength computed from top 11 players by", score_col)
print("Nations:", nation_xi.shape[0])

display(nation_xi.head(25))

# Save
out_path_xi = "nation_rankings_starting_xi_top11.csv"
nation_xi.to_csv(out_path_xi, index=False)
print("Saved:", out_path_xi)


Nation Starting XI Strength computed from top 11 players by weighted_score_norm2
Nations: 37


,Nation_code,n_xi_players,xi_score_sum,xi_score_mean,xi_score_median,xi_score_min,xi_score_max,xi_minutes90_sum,starting_xi_players
0,FRA,11,559.147562,50.831597,48.977293,44.306620,67.750659,197.6,"Ousmane Dembélé, Kingsley Coman, Bradley Barco..."
1,ESP,11,554.364806,50.396801,50.244641,47.150408,56.031176,241.1,"Rodrigo Riquelme, Isco, Luis Alberto, Álex Gri..."
2,GER,11,541.143168,49.194833,46.361525,41.562544,79.415455,262.2,"Florian Wirtz, Jonas Hofmann, Joshua Kimmich, ..."
3,ENG,11,527.363533,47.942139,46.767619,40.177069,58.002068,218.1,"James Maddison, Jude Bellingham, Bukayo Saka, ..."
4,NED,11,481.369069,43.760824,42.034286,32.042258,67.470000,226.9,"Jeremie Frimpong, Xavi Simons, Teun Koopmeiner..."
5,BRA,11,475.586464,43.235133,40.155502,37.270340,57.910866,231.6,"Samuel Lino, Gabriel Martinelli, Vinicius Júni..."
6,POR,11,466.123389,42.374854,42.249930,38.038866,49.300303,221.2,"Raphaël Guerreiro, João Cancelo, Vitinha, Rafa..."
7,MAR,11,436.435305,39.675937,34.303415,23.186923,75.423023,169.7,"Achraf Hakimi, Noussair Mazraoui, Azzedine Oun..."
8,BEL,11,417.783629,37.980330,31.725714,25.507977,69.754118,204.7,"Kevin De Bruyne, Jeremy Doku, Charles De Ketel..."
9,ARG,11,403.688364,36.698942,35.662365,32.311455,44.294113,279.2,"Rodrigo De Paul, Nahuel Molina, Enzo Fernández..."


Saved: nation_rankings_starting_xi_top11.csv


In [45]:
# Compute nation "Star Player Impact" = average score of top 3 players by weighted_score_norm2
import pandas as pd

score_col = "weighted_score_norm2"
assert score_col in final_rankings.columns, f"Missing {score_col} in final_rankings"

base_star = final_rankings[["Nation_code", "Player", "Pos_group", "Playing Time 90s", score_col]].copy()

# Sort players within each nation by score (desc), take top 3
star3 = (
    base_star.sort_values(["Nation_code", score_col], ascending=[True, False])
             .groupby("Nation_code")
             .head(3)
             .copy()
)

# Aggregate star player impact
nation_star_impact = (
    star3.groupby("Nation_code", as_index=False)
         .agg(
             n_star_players=("Player", "count"),
             star3_avg_score=(score_col, "mean"),
             star3_sum_score=(score_col, "sum"),
             star3_min_score=(score_col, "min"),
             star3_max_score=(score_col, "max"),
             star3_minutes90_sum=("Playing Time 90s", "sum"),
         )
)

# Add a readable list of the 3 players (with scores)
star3_list = (
    star3.assign(player_with_score=lambda d: d["Player"] + " (" + d[score_col].round(2).astype(str) + ")")
         .groupby("Nation_code")["player_with_score"]
         .apply(lambda s: ", ".join(s.tolist()))
         .rename("star3_players")
         .reset_index()
)

nation_star_impact = nation_star_impact.merge(star3_list, on="Nation_code", how="left")

# Sort by star player impact (avg of top 3)
nation_star_impact = nation_star_impact.sort_values(
    ["star3_avg_score", "star3_max_score"], ascending=False
).reset_index(drop=True)

print("Nation Star Player Impact computed as mean(top 3 players by", score_col + ")")
print("Nations:", nation_star_impact.shape[0])

display(nation_star_impact.head(25))

# Save
out_path_star = "nation_rankings_star_player_impact_top3_avg.csv"
nation_star_impact.to_csv(out_path_star, index=False)
print("Saved:", out_path_star)


Nation Star Player Impact computed as mean(top 3 players by weighted_score_norm2)
Nations: 37


,Nation_code,n_star_players,star3_avg_score,star3_sum_score,star3_min_score,star3_max_score,star3_minutes90_sum,star3_players
0,GER,3,60.863370,182.590110,51.210165,79.415455,75.1,"Florian Wirtz (79.42), Jonas Hofmann (51.96), ..."
1,FRA,3,59.591272,178.773817,51.043158,67.750659,44.3,"Ousmane Dembélé (67.75), Kingsley Coman (59.98..."
2,BEL,3,59.416950,178.250850,41.909444,69.754118,53.8,"Kevin De Bruyne (69.75), Jeremy Doku (66.59), ..."
3,NED,3,57.900324,173.700971,47.148767,67.470000,83.7,"Jeremie Frimpong (67.47), Xavi Simons (59.08),..."
4,MAR,3,56.677168,170.031503,45.322239,75.423023,48.2,"Achraf Hakimi (75.42), Noussair Mazraoui (49.2..."
5,ENG,3,55.775629,167.326886,52.465247,58.002068,81.8,"James Maddison (58.0), Jude Bellingham (56.86)..."
6,ESP,3,54.165454,162.496363,50.728132,56.031176,68.5,"Rodrigo Riquelme (56.03), Isco (55.74), Luis A..."
7,BRA,3,52.837245,158.511734,50.059082,57.910866,66.2,"Samuel Lino (57.91), Gabriel Martinelli (50.54..."
8,POR,3,47.665484,142.996451,45.615932,49.300303,64.6,"Raphaël Guerreiro (49.3), João Cancelo (48.08)..."
9,JPN,3,46.575230,139.725691,40.317155,53.121515,63.9,"Kaoru Mitoma (53.12), Takumi Minamino (46.29),..."


Saved: nation_rankings_star_player_impact_top3_avg.csv


In [46]:
# Compute nation "Squad Depth" = average score of players ranked 12–23 by weighted_score_norm2
import pandas as pd

score_col = "weighted_score_norm2"
assert score_col in final_rankings.columns, f"Missing {score_col} in final_rankings"

base_depth = final_rankings[["Nation_code", "Player", "Pos_group", "Playing Time 90s", score_col]].copy()

# Rank players within each nation by score (desc)
base_depth["rank_in_nation"] = (
    base_depth.groupby("Nation_code")[score_col]
              .rank(method="first", ascending=False)
)

# Keep ranks 12–23 inclusive
bench_12_23 = base_depth[(base_depth["rank_in_nation"] >= 12) & (base_depth["rank_in_nation"] <= 23)].copy()

nation_depth = (
    bench_12_23.groupby("Nation_code", as_index=False)
               .agg(
                   n_depth_players=("Player", "count"),
                   depth12_23_avg_score=(score_col, "mean"),
                   depth12_23_sum_score=(score_col, "sum"),
                   depth12_23_min_score=(score_col, "min"),
                   depth12_23_max_score=(score_col, "max"),
                   depth12_23_minutes90_sum=("Playing Time 90s", "sum"),
               )
)

# Add a readable list (up to 12 players) with scores
bench_list = (
    bench_12_23.sort_values(["Nation_code", "rank_in_nation"], ascending=[True, True])
               .assign(player_with_score=lambda d: d["Player"] + " (" + d[score_col].round(2).astype(str) + ")")
               .groupby("Nation_code")["player_with_score"]
               .apply(lambda s: ", ".join(s.tolist()))
               .rename("depth12_23_players")
               .reset_index()
)

nation_depth = nation_depth.merge(bench_list, on="Nation_code", how="left")

# Sort by depth average score
nation_depth = nation_depth.sort_values(
    ["depth12_23_avg_score", "depth12_23_min_score"], ascending=False
).reset_index(drop=True)

print("Nation Squad Depth computed as mean(players ranked 12–23 by", score_col + ")")
print("Nations with >=1 depth player (rank 12–23):", nation_depth.shape[0])

display(nation_depth.head(25))

# Save
out_path_depth = "nation_rankings_squad_depth_12_23_avg.csv"
nation_depth.to_csv(out_path_depth, index=False)
print("Saved:", out_path_depth)


Nation Squad Depth computed as mean(players ranked 12–23 by weighted_score_norm2)
Nations with >=1 depth player (rank 12–23): 23


,Nation_code,n_depth_players,depth12_23_avg_score,depth12_23_sum_score,depth12_23_min_score,depth12_23_max_score,depth12_23_minutes90_sum,depth12_23_players
0,FRA,12,41.875015,502.500182,40.452353,43.717383,249.7,"Ruben Aguilar (43.72), Rémy Cabella (43.18), D..."
1,ESP,12,41.439344,497.272127,38.301570,47.114885,203.3,"Juanlu Sánchez (47.11), Alfonso Pedraza (45.3)..."
2,GER,12,38.757998,465.095975,36.282376,41.305897,249.8,"Jamal Musiala (41.31), Marius Wolf (40.46), Mi..."
3,ENG,12,36.768141,441.217694,34.949709,39.857097,290.5,"Kieran Trippier (39.86), Kyle Walker (39.77), ..."
4,BRA,12,33.997185,407.966224,29.367917,36.815738,258.0,"Ângelo Borges (36.82), João Pedro (36.51), Jun..."
5,POR,12,30.698017,368.376205,23.626338,37.958517,283.3,"Tiago santos (37.96), Diogo Jota (35.65), Gonç..."
6,ARG,12,27.260127,327.121518,22.941711,30.470719,254.1,"Óscar Trejo (30.47), Ángel Correa (30.35), Val..."
7,NED,12,25.115334,301.384003,21.964030,31.573497,250.3,"Cody Gakpo (31.57), Joël Veltman (27.82), Rick..."
8,BEL,12,20.784837,249.418049,13.211453,25.140980,238.3,"Thomas Foket (25.14), Timothy Castagne (24.7),..."
9,AUT,12,19.446864,233.362362,9.644809,25.977015,251.7,"Stefan Posch (25.98), Florian Grillitsch (25.0..."


Saved: nation_rankings_squad_depth_12_23_avg.csv


In [47]:
# Compute player-based nation strength as a weighted combo of Starting XI, Star impact, and Squad depth
import numpy as np
import pandas as pd

score_col = "weighted_score_norm2"
base = final_rankings[["Nation_code", "Player", score_col]].copy()

# Rank players within nation by player score (desc)
base["rank_in_nation"] = base.groupby("Nation_code")[score_col].rank(method="first", ascending=False)

# Helper: safe mean
def safe_mean(s: pd.Series) -> float:
    s = pd.to_numeric(s, errors="coerce")
    s = s[np.isfinite(s)]
    return float(s.mean()) if len(s) else np.nan

# Compute components per nation
components = []
for nation, g in base.groupby("Nation_code"):
    g = g.sort_values(score_col, ascending=False)

    top11_mean = safe_mean(g.head(11)[score_col])
    top3_mean = safe_mean(g.head(3)[score_col])

    # Depth: ranks 12-23 inclusive (may be empty)
    depth = g[(g["rank_in_nation"] >= 12) & (g["rank_in_nation"] <= 23)]
    depth_mean = safe_mean(depth[score_col])

    player_based_score = 0.50 * top11_mean + 0.25 * top3_mean + 0.25 * depth_mean

    components.append({
        "Nation_code": nation,
        "n_players": int(len(g)),
        "top11_mean": top11_mean,
        "top3_mean": top3_mean,
        "depth12_23_mean": depth_mean,
        "player_based_score": player_based_score,
    })

player_based_nation_strength = pd.DataFrame(components)
player_based_nation_strength = player_based_nation_strength.sort_values("player_based_score", ascending=False).reset_index(drop=True)

print("Player-based nation strength computed using:")
print("0.50*mean(top11) + 0.25*mean(top3) + 0.25*mean(ranks 12-23)")
print("Note: nations with <23 players may have NaN depth; those will yield NaN player_based_score unless handled.")

display(player_based_nation_strength.head(25))

out_path = "nation_player_based_strength_weighted_50_25_25.csv"
player_based_nation_strength.to_csv(out_path, index=False)
print("Saved:", out_path)


Player-based nation strength computed using:
0.50*mean(top11) + 0.25*mean(top3) + 0.25*mean(ranks 12-23)
Note: nations with <23 players may have NaN depth; those will yield NaN player_based_score unless handled.


,Nation_code,n_players,top11_mean,top3_mean,depth12_23_mean,player_based_score
0,FRA,187,50.831597,59.591272,41.875015,50.782370
1,GER,157,49.194833,60.863370,38.757998,49.502759
2,ESP,226,50.396801,54.165454,41.439344,49.099600
3,ENG,113,47.942139,55.775629,36.768141,47.107012
4,BRA,70,43.235133,52.837245,33.997185,43.326174
5,NED,43,43.760824,57.900324,25.115334,42.634327
6,POR,41,42.374854,47.665484,30.698017,40.778302
7,BEL,31,37.980330,59.416950,20.784837,39.040612
8,MAR,21,39.675937,56.677168,12.503362,37.133101
9,ARG,45,36.698942,42.014804,27.260127,35.668204


Saved: nation_player_based_strength_weighted_50_25_25.csv


In [57]:
# Prepare nation strength features for pairwise logistic-regression matchup modeling
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.linear_model import LogisticRegression

# Base nation strength table computed earlier
pb = player_based_nation_strength.copy()

# Keep only nations in allowed list (by Nation_code available in pb)
# Note: allowed_country_names are names; pb is Nation_code. We'll use Nation_code list present in pb.

# Feature columns (numeric) from pb
feature_cols_pb = [
    "top11_mean",
    "top3_mean",
    "depth12_23_mean",
    "player_based_score",
]

# Coerce numeric
for c in feature_cols_pb:
    pb[c] = pd.to_numeric(pb[c], errors="coerce")

# Drop nations where key feature is missing (depth can be NaN for small nations)
# We'll keep rows with player_based_score present; for depth NaN we can impute later if needed.
print("player_based_nation_strength shape:", pb.shape)
print("Missing rates:")
print(pb[feature_cols_pb].isna().mean().sort_values(ascending=False))

# Build all unordered matchups (A,B) from available nations
nations = pb["Nation_code"].dropna().unique().tolist()
matchups = list(combinations(sorted(nations), 2))
matchups_df = pd.DataFrame(matchups, columns=["team_A", "team_B"])

# Merge features for A and B
A = pb[["Nation_code"] + feature_cols_pb].rename(columns={"Nation_code": "team_A"})
B = pb[["Nation_code"] + feature_cols_pb].rename(columns={"Nation_code": "team_B"})
matchups_df = matchups_df.merge(A, on="team_A", how="left", suffixes=(None, None))
matchups_df = matchups_df.merge(B, on="team_B", how="left", suffixes=("_A", "_B"))

# Ensure suffixes are correct
# After merges, columns will be: team_A, team_B, top11_mean, ... then top11_mean_B etc depending on pandas behavior.
# We'll standardize to explicit _A/_B names.
rename_map = {}
for c in feature_cols_pb:
    if c in matchups_df.columns:
        rename_map[c] = c + "_A"
    if c + "_B" in matchups_df.columns:
        # already ok
        pass
    elif c in matchups_df.columns and c + "_A" not in matchups_df.columns:
        pass
matchups_df = matchups_df.rename(columns=rename_map)

# If B columns didn't get _B suffix due to name collision rules, fix them
for c in feature_cols_pb:
    if c in matchups_df.columns and c + "_A" in matchups_df.columns:
        # ambiguous; skip
        pass

# Create difference features (A - B)
for c in feature_cols_pb:
    ca, cb = c + "_A", c + "_B"
    if ca in matchups_df.columns and cb in matchups_df.columns:
        matchups_df[c + "_diff"] = matchups_df[ca] - matchups_df[cb]

diff_cols = [c + "_diff" for c in feature_cols_pb if c + "_diff" in matchups_df.columns]
print("Matchups:", len(matchups_df), "| diff feature cols:", diff_cols)

display(matchups_df.head())

print("\nNOTE: To fit LogisticRegression we need historical match outcomes (a target y indicating whether team_A beat team_B).\n"
      "If you provide a match results dataset or specify a proxy target (e.g., higher ELO wins), we can train and then output win probabilities for all pairings.")


player_based_nation_strength shape: (37, 6)
Missing rates:
player_based_score    0.378378
depth12_23_mean       0.378378
top3_mean             0.000000
top11_mean            0.000000
dtype: float64
Matchups: 666 | diff feature cols: ['top11_mean_diff', 'top3_mean_diff', 'depth12_23_mean_diff', 'player_based_score_diff']


,team_A,team_B,top11_mean_A,top3_mean_A,depth12_23_mean_A,player_based_score_A,top11_mean_B,top3_mean_B,depth12_23_mean_B,player_based_score_B,top11_mean_diff,top3_mean_diff,depth12_23_mean_diff,player_based_score_diff
0,ALG,ARG,27.921551,34.226474,-0.090361,22.494804,36.698942,42.014804,27.260127,35.668204,-8.777391,-7.788330,-27.350488,-13.173400
1,ALG,AUT,27.921551,34.226474,-0.090361,22.494804,35.011643,45.646206,19.446864,33.779089,-7.090092,-11.419732,-19.537225,-11.284285
2,ALG,BEL,27.921551,34.226474,-0.090361,22.494804,37.980330,59.416950,20.784837,39.040612,-10.058779,-25.190476,-20.875199,-16.545808
3,ALG,BRA,27.921551,34.226474,-0.090361,22.494804,43.235133,52.837245,33.997185,43.326174,-15.313582,-18.610771,-34.087547,-20.831371
4,ALG,CAN,27.921551,34.226474,-0.090361,22.494804,30.504456,30.504456,NaN,NaN,-2.582905,3.722018,NaN,NaN



NOTE: To fit LogisticRegression we need historical match outcomes (a target y indicating whether team_A beat team_B).
If you provide a match results dataset or specify a proxy target (e.g., higher ELO wins), we can train and then output win probabilities for all pairings.


In [58]:
# Create a proxy-trained logistic regression model and compute head-to-head win probabilities
# NOTE: Without historical match results, we train on a proxy label: team_A wins if its player_based_score > team_B.
# This produces a smooth probability mapping of strength-differences, NOT an empirically validated win model.

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Start from matchups_df created earlier (all combinations) and pb (nation strength features)
req_diff_cols = [
    'top11_mean_diff',
    'top3_mean_diff',
    'depth12_23_mean_diff',
    'player_based_score_diff',
]

# Use the most complete subset for modeling
model_df = matchups_df.copy()

# Keep rows where core diff features are available
# Depth and player_based_score can be NaN for nations with <23 eligible players.
# We'll fit on rows with non-missing player_based_score_diff and top11/top3 diffs.
core = ['top11_mean_diff', 'top3_mean_diff', 'player_based_score_diff']
model_df = model_df.dropna(subset=core).copy()

# If depth is missing, set it to 0 difference (equivalent depth signal absent)
if 'depth12_23_mean_diff' in model_df.columns:
    model_df['depth12_23_mean_diff'] = model_df['depth12_23_mean_diff'].fillna(0.0)

X = model_df[req_diff_cols].astype(float)

# Proxy target: A wins if player_based_score_diff > 0
# (ties -> 0)
y = (model_df['player_based_score_diff'] > 0).astype(int)

# Fit logistic regression (no regularization tuning here; keep interpretable)
clf = LogisticRegression(max_iter=500, solver='lbfgs')
clf.fit(X, y)

# Compute probabilities for ALL matchups (including ones with missing depth diff)
pred_df = matchups_df.copy()
pred_df = pred_df.dropna(subset=core).copy()
if 'depth12_23_mean_diff' in pred_df.columns:
    pred_df['depth12_23_mean_diff'] = pred_df['depth12_23_mean_diff'].fillna(0.0)

Xp = pred_df[req_diff_cols].astype(float)

pA = clf.predict_proba(Xp)[:, 1]
pred_df['P_team_A_wins'] = pA
pred_df['P_team_B_wins'] = 1 - pA

# Make a tidy output
matchup_probs = (
    pred_df[['team_A','team_B','P_team_A_wins','P_team_B_wins'] + req_diff_cols]
    .sort_values('P_team_A_wins', ascending=False)
    .reset_index(drop=True)
)

print('LogisticRegression trained on proxy target: team_A wins if player_based_score_A > player_based_score_B')
print('Training rows used:', len(model_df), 'out of', len(matchups_df))
print('Coefficients (higher -> increases P(team_A wins))')
coef = pd.Series(clf.coef_.ravel(), index=req_diff_cols).sort_values(ascending=False)
display(coef.to_frame('coef'))

print('\nTop 20 most lopsided matchups (by P_team_A_wins):')
display(matchup_probs.head(20))

# Save
out_path = 'nation_matchup_probabilities_proxy_logreg.csv'
matchup_probs.to_csv(out_path, index=False)
print('Saved:', out_path)


LogisticRegression trained on proxy target: team_A wins if player_based_score_A > player_based_score_B
Training rows used: 253 out of 666
Coefficients (higher -> increases P(team_A wins))


,coef
top11_mean_diff,0.990116
player_based_score_diff,0.871911
top3_mean_diff,0.785808
depth12_23_mean_diff,0.721603



Top 20 most lopsided matchups (by P_team_A_wins):


,team_A,team_B,P_team_A_wins,P_team_B_wins,top11_mean_diff,top3_mean_diff,depth12_23_mean_diff,player_based_score_diff
0,ARG,COL,1.0,0.0,11.543589,7.220129,19.817514,12.531205
1,AUT,COL,1.0,0.0,9.856290,10.851531,12.004251,10.642090
2,BRA,COL,1.0,0.0,18.079780,18.042569,26.554573,20.189176
3,BRA,GHA,1.0,0.0,14.644787,15.730868,21.606858,16.656825
4,BRA,JPN,1.0,0.0,8.562628,6.262014,24.314643,11.925478
5,BEL,SEN,1.0,0.0,7.864462,21.060449,5.836658,10.656508
6,BEL,URU,1.0,0.0,7.975797,22.307222,5.968794,11.056903
7,BRA,CIV,1.0,0.0,12.950404,16.040634,20.291917,15.558340
8,BEL,USA,1.0,0.0,8.932717,23.000330,11.072819,12.984646
9,BEL,GHA,1.0,0.0,9.389984,22.310573,8.394510,12.371263


Saved: nation_matchup_probabilities_proxy_logreg.csv


In [61]:
# Recompute matchup probabilities for ALL nation pairings by imputing missing depth and player_based_score
import numpy as np
import pandas as pd
from itertools import permutations

# Start from player_based_nation_strength
pb_full = player_based_nation_strength.copy()

# Impute missing depth with 0 (no evidence of depth); then recompute player_based_score where missing
# player_based_score = 0.50*top11_mean + 0.25*top3_mean + 0.25*depth12_23_mean
pb_full["depth12_23_mean"] = pd.to_numeric(pb_full["depth12_23_mean"], errors="coerce").fillna(0.0)
for c in ["top11_mean", "top3_mean"]:
    pb_full[c] = pd.to_numeric(pb_full[c], errors="coerce")

pb_full["player_based_score_imputed"] = (
    0.50 * pb_full["top11_mean"] +
    0.25 * pb_full["top3_mean"] +
    0.25 * pb_full["depth12_23_mean"]
)

# Use imputed score for matchup features
feature_cols_pb2 = ["top11_mean", "top3_mean", "depth12_23_mean", "player_based_score_imputed"]

# All ordered permutations A vs B (A!=B)
teams = sorted(pb_full["Nation_code"].dropna().unique().tolist())
perm = [(a, b) for a, b in permutations(teams, 2)]
matchups_all = pd.DataFrame(perm, columns=["team_A", "team_B"])

A2 = pb_full[["Nation_code"] + feature_cols_pb2].rename(columns={"Nation_code": "team_A"})
B2 = pb_full[["Nation_code"] + feature_cols_pb2].rename(columns={"Nation_code": "team_B"})

matchups_all = matchups_all.merge(A2, on="team_A", how="left")
matchups_all = matchups_all.merge(B2, on="team_B", how="left", suffixes=("_A", "_B"))

# Difference features (A - B), aligned with training columns in clf
# Difference features (A - B)
# NOTE: because of the merge order + suffixes, team_A columns end with _A and team_B columns end with _B.
matchups_all["top11_mean_diff"] = matchups_all["top11_mean_A"] - matchups_all["top11_mean_B"]
matchups_all["top3_mean_diff"] = matchups_all["top3_mean_A"] - matchups_all["top3_mean_B"]
matchups_all["depth12_23_mean_diff"] = matchups_all["depth12_23_mean_A"] - matchups_all["depth12_23_mean_B"]
matchups_all["player_based_score_diff"] = (
    matchups_all["player_based_score_imputed_A"] - matchups_all["player_based_score_imputed_B"]
)

# Predict with the existing trained LogisticRegression (clf) from cell 19
X_all = matchups_all[[
    "top11_mean_diff",
    "top3_mean_diff",
    "depth12_23_mean_diff",
    "player_based_score_diff",
]].astype(float)

pA_all = clf.predict_proba(X_all)[:, 1]
matchups_all["P_team_A_wins"] = pA_all
matchups_all["P_team_B_wins"] = 1 - pA_all

matchup_probs_all = (
    matchups_all[["team_A", "team_B", "P_team_A_wins", "P_team_B_wins"]]
    .sort_values(["P_team_A_wins"], ascending=False)
    .reset_index(drop=True)
)

print("All ordered matchups:", len(matchup_probs_all), "(should be n*(n-1)) with n=", len(teams))
print("Columns present (sanity check):", [c for c in ["top11_mean_A","top11_mean_B","player_based_score_imputed_A","player_based_score_imputed_B"] if c in matchups_all.columns])
display(matchup_probs_all.head(20))

out_path_all = "nation_matchup_probabilities_proxy_logreg_ALL_pairs.csv"
matchup_probs_all.to_csv(out_path_all, index=False)
print("Saved:", out_path_all)


All ordered matchups: 1332 (should be n*(n-1)) with n= 37
Columns present (sanity check): ['top11_mean_A', 'top11_mean_B', 'player_based_score_imputed_A', 'player_based_score_imputed_B']


,team_A,team_B,P_team_A_wins,P_team_B_wins
0,NED,USA,1.0,0.0
1,NED,URU,1.0,0.0
2,NED,TUN,1.0,0.0
3,NED,SUI,1.0,0.0
4,NED,SEN,1.0,0.0
5,NED,SCO,1.0,0.0
6,NED,RSA,1.0,0.0
7,NED,PAR,1.0,0.0
8,NED,PAN,1.0,0.0
9,NED,NZL,1.0,0.0


Saved: nation_matchup_probabilities_proxy_logreg_ALL_pairs.csv


### How the head-to-head win probabilities were calculated

The probabilities in `matchup_probs` / `matchup_probs_all` come from a **logistic regression model** (`clf`) applied to **pairwise feature differences** between two nations.

#### 1) Start from nation-level strength features
We used `player_based_nation_strength` (37 nations) with these columns:
- `top11_mean` = mean score of the nation’s top 11 players
- `top3_mean` = mean score of the nation’s top 3 players
- `depth12_23_mean` = mean score of players ranked 12–23 (can be missing for small nations)
- `player_based_score` = `0.50*top11_mean + 0.25*top3_mean + 0.25*depth12_23_mean`

All of these are derived from the player-level `weighted_score_norm2` in `final_rankings`.

#### 2) Build matchup rows and “A − B” difference features
For every matchup (Team A vs Team B), we created **difference features**:
- `top11_mean_diff = top11_mean_A - top11_mean_B`
- `top3_mean_diff = top3_mean_A - top3_mean_B`
- `depth12_23_mean_diff = depth12_23_mean_A - depth12_23_mean_B`
- `player_based_score_diff = player_based_score_A - player_based_score_B`

This is done so the model learns a single set of coefficients that applies to *any* matchup direction.

#### 3) Training target: proxy win/loss label (because no real match results were provided)
Because we **did not have historical match outcomes** (a true `y` label), we trained the logistic regression on a **proxy target**:

- `y = 1` if `player_based_score_diff > 0` (Team A is “stronger” than Team B)
- `y = 0` otherwise

So the model is **not** learning from real match results; it’s learning a smooth mapping from strength-differences to a probability scale.

#### 4) Fit a LogisticRegression
We fit:

\n\n`P(A wins) = sigmoid( b0 + b1*top11_mean_diff + b2*top3_mean_diff + b3*depth12_23_mean_diff + b4*player_based_score_diff )`\n\n
where `sigmoid(z) = 1/(1+exp(-z))`.

In scikit-learn terms:
- `clf = LogisticRegression(...)`
- `clf.fit(X, y)`

#### 5) Convert to probabilities via `predict_proba`
For each matchup row, we computed:
- `P_team_A_wins = clf.predict_proba(X_matchup)[..., 1]`
- `P_team_B_wins = 1 - P_team_A_wins`

#### 6) Handling missing depth / small nations (cell 20)
Some nations have no ranks 12–23, so `depth12_23_mean` is missing and therefore `player_based_score` can also be missing.

To produce probabilities for **all ordered pairs** in `matchup_probs_all`, we imputed:
- `depth12_23_mean = 0` when missing
- recomputed `player_based_score_imputed = 0.50*top11_mean + 0.25*top3_mean + 0.25*depth12_23_mean`

Then we used `player_based_score_imputed_diff` in the prediction table.

---

If you provide **real match results** (or an external rating like FIFA/ELO over time with outcomes), we can train a logistic model on actual win/loss targets and the probabilities will become empirically grounded instead of proxy-calibrated.

In [62]:
# Diagnose why matchup probabilities saturate at 0/1
import numpy as np
import pandas as pd

# Use the full ordered matchups (built in cell 20)
assert 'matchups_all' in globals() and isinstance(matchups_all, pd.DataFrame)

# Compute logit (decision function) and probability summary
X_all = matchups_all[[
    "top11_mean_diff",
    "top3_mean_diff",
    "depth12_23_mean_diff",
    "player_based_score_diff",
]].astype(float)

logit = clf.decision_function(X_all)
proba = clf.predict_proba(X_all)[:, 1]

print("Predicted probability summary:")
print(pd.Series(proba).describe(percentiles=[0.001, 0.01, 0.05, 0.5, 0.95, 0.99, 0.999]))

print("\nDecision function (logit) summary:")
print(pd.Series(logit).describe(percentiles=[0.001, 0.01, 0.05, 0.5, 0.95, 0.99, 0.999]))

# Count how many are extremely close to 0 or 1
print("\nShare near 0 (<=1%):", float((proba <= 0.01).mean()))
print("Share near 1 (>=99%):", float((proba >= 0.99).mean()))

# Inspect feature scale
print("\nDiff feature scale snapshot:")
desc = X_all.describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).T
print(desc[["mean","std","min","1%","5%","50%","95%","99%","max"]])


Predicted probability summary:
count    1.332000e+03
mean     5.038254e-01
std      4.935599e-01
min      2.258751e-57
0.1%     8.736413e-56
1%       2.502724e-52
5%       1.558632e-40
50%      6.417277e-01
95%      1.000000e+00
99%      1.000000e+00
99.9%    1.000000e+00
max      1.000000e+00
dtype: float64

Decision function (logit) summary:
count    1332.000000
mean        0.582873
std        54.261584
min      -130.432538
0.1%     -126.812008
1%       -118.913061
5%        -91.659616
50%         0.582873
95%        92.825362
99%       120.078808
99.9%     127.977755
max       131.598285
dtype: float64

Share near 0 (<=1%): 0.4617117117117117
Share near 1 (>=99%): 0.47072072072072074

Diff feature scale snapshot:
                                 mean        std        min         1%  \
top11_mean_diff          0.000000e+00  13.531061 -36.607959 -30.344822   
top3_mean_diff           0.000000e+00  18.672050 -42.724186 -39.490795   
depth12_23_mean_diff     3.414019e-16  19.296617 -41

In [63]:
# Refit logistic regression on standardized matchup diff features to avoid 0/1 saturation
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Use matchups_all (ordered pairs) and the earlier proxy label rule
X_cols = [
    "top11_mean_diff",
    "top3_mean_diff",
    "depth12_23_mean_diff",
    # player_based_score_diff is a linear combo of the other 3 (with our imputation),
    # so exclude it to reduce redundancy / extreme coefficients.
]

X_train = matchups_all[X_cols].astype(float).copy()

y_train = (matchups_all["player_based_score_diff"] > 0).astype(int)

# Standardize features then fit logistic regression with stronger regularization (smaller C)
cal_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=2000, solver="lbfgs", C=0.5))
])
cal_clf.fit(X_train, y_train)

# Predict probabilities for all ordered pairs
pA = cal_clf.predict_proba(X_train)[:, 1]
matchup_probs_all_cal = matchups_all[["team_A", "team_B"]].copy()
matchup_probs_all_cal["P_team_A_wins"] = pA
matchup_probs_all_cal["P_team_B_wins"] = 1 - pA

# Add a rounded percent view for readability
matchup_probs_all_cal["P_team_A_wins_pct"] = (100 * matchup_probs_all_cal["P_team_A_wins"]).round(1)
matchup_probs_all_cal["P_team_B_wins_pct"] = (100 * matchup_probs_all_cal["P_team_B_wins"]).round(1)

# Sort by closest to coinflip to see calibration quality too
matchup_probs_all_cal["abs_diff_from_50"] = (matchup_probs_all_cal["P_team_A_wins"] - 0.5).abs()

print("Calibrated probability summary:")
print(matchup_probs_all_cal["P_team_A_wins"].describe(percentiles=[0.01,0.05,0.5,0.95,0.99]))

print("\nMost lopsided (A strongest):")
display(matchup_probs_all_cal.sort_values("P_team_A_wins", ascending=False).head(20))

print("\nMost even matchups (closest to 50/50):")
display(matchup_probs_all_cal.sort_values("abs_diff_from_50", ascending=True).head(20))

out_path = "nation_matchup_probabilities_proxy_logreg_ALL_pairs_calibrated.csv"
matchup_probs_all_cal.drop(columns=["abs_diff_from_50"]).to_csv(out_path, index=False)
print("\nSaved:", out_path)


Calibrated probability summary:
count    1.332000e+03
mean     5.000000e-01
std      4.628786e-01
min      1.240441e-11
1%       1.247420e-10
5%       2.394659e-08
50%      5.000000e-01
95%      1.000000e+00
99%      1.000000e+00
max      1.000000e+00
Name: P_team_A_wins, dtype: float64

Most lopsided (A strongest):


,team_A,team_B,P_team_A_wins,P_team_B_wins,P_team_A_wins_pct,P_team_B_wins_pct,abs_diff_from_50
525,FRA,MEX,1.0,1.240430e-11,100.0,0.0,0.5
512,FRA,CPV,1.0,2.631406e-11,100.0,0.0,0.5
561,GER,MEX,1.0,2.720602e-11,100.0,0.0,0.5
539,FRA,UZB,1.0,3.068301e-11,100.0,0.0,0.5
520,FRA,HAI,1.0,4.066791e-11,100.0,0.0,0.5
489,ESP,MEX,1.0,4.089329e-11,100.0,0.0,0.5
548,GER,CPV,1.0,5.771339e-11,100.0,0.0,0.5
575,GER,UZB,1.0,6.729528e-11,100.0,0.0,0.5
532,FRA,RSA,1.0,7.475287e-11,100.0,0.0,0.5
528,FRA,NZL,1.0,7.665202e-11,100.0,0.0,0.5



Most even matchups (closest to 50/50):


,team_A,team_B,P_team_A_wins,P_team_B_wins,P_team_A_wins_pct,P_team_B_wins_pct,abs_diff_from_50
401,EGY,CAN,0.498229,0.501771,49.8,50.2,0.001771
190,CAN,EGY,0.501771,0.498229,50.2,49.8,0.001771
1069,RSA,NZL,0.493728,0.506272,49.4,50.6,0.006272
928,NZL,RSA,0.506272,0.493728,50.6,49.4,0.006272
10,ALG,EGY,0.479414,0.520586,47.9,52.1,0.020586
396,EGY,ALG,0.520586,0.479414,52.1,47.9,0.020586
4,ALG,CAN,0.477646,0.522354,47.8,52.2,0.022354
180,CAN,ALG,0.522354,0.477646,52.2,47.8,0.022354
261,COL,ECU,0.474251,0.525749,47.4,52.6,0.025749
367,ECU,COL,0.525749,0.474251,52.6,47.4,0.025749



Saved: nation_matchup_probabilities_proxy_logreg_ALL_pairs_calibrated.csv


In [64]:
# Lookup the calibrated head-to-head prediction for Germany vs France (both directions)
import pandas as pd

# Ensure the calibrated matchup table exists
assert "matchup_probs_all_cal" in globals(), "matchup_probs_all_cal not found; run the calibrated matchup cell first."

# Extract the two directed matchups
gef = matchup_probs_all_cal[(matchup_probs_all_cal["team_A"] == "GER") & (matchup_probs_all_cal["team_B"] == "FRA")].copy()
fge = matchup_probs_all_cal[(matchup_probs_all_cal["team_A"] == "FRA") & (matchup_probs_all_cal["team_B"] == "GER")].copy()

out = pd.concat([gef, fge], ignore_index=True)

# Keep only the relevant columns for display
cols = [
    "team_A","team_B",
    "P_team_A_wins","P_team_B_wins",
    "P_team_A_wins_pct","P_team_B_wins_pct",
]
out = out[cols].sort_values(["team_A","team_B"]).reset_index(drop=True)

out

,team_A,team_B,P_team_A_wins,P_team_B_wins,P_team_A_wins_pct,P_team_B_wins_pct
0,FRA,GER,0.686839,0.313161,68.7,31.3
1,GER,FRA,0.313161,0.686839,31.3,68.7
